# MA6202: Laboratorio de Ciencia de Datos

**Profesor: Nicolás Caro**

**10/06/2020 - C11 S7**

# Ingeniería de Características y Algoritmos de Aprendizaje Automático

La práctica de la ciencia de datos se fundamenta en la obtención, exploración, limpieza, tratamiento y transformación de la información contenida en distintas fuentes de datos. Tales procedimientos constituyen gran parte del desarrollo de proyectos basados datos y su finalidad es la facilitar la obtención de conocimiento. Este último paso se lleva a cabo por medio de sistemas de aprendizaje automático. En general esto se puede resumir en las siguientes etapas:

1. **Obtención de la información**: Corresponde a la recolección de datos referentes al fenómeno de interés. Tal información puede ser tanto estructurada como no estructurada.

2. **Preprocesamiento y exploración**: Una vez reunida una porción de información significativa, se procede a explorarla y procesarla. En este apartado se aplican métodos de reducción de dimensionalidad, escalamiento, muestreo (conjuntos de entrenamiento y test), selección de características y transformación. 

3. **Aprendizaje**: Cuando los datos han sido estudiados y transformados de buena manera, se procede a la aplicación de modelos de aprendizaje de máquinas. Acá se estudian métricas de rendimiento y se optimizan hiperparámetros. 

4. **Evaluación**: Seleccionado un modelo de optimo, se generan esquemas de evaluación sobre conjuntos de datos no observados pero representativos del fenómeno estudiado, esto permite adquirir nociones sobre el comportamiento del sistema modelado fuera del conjunto de entrenamiento. 

5. **Producción**: Finalmente se genera un entorno en cual el modelo obtenido es utilizado para la obtención de información, elaboración de predicciones, clasificaciones o perfilamientos en función de la materia estudiada. 

Estos procesos son interdependientes y no presentan un orden lineal, más bien, se entrelazan circulando de manera natural entre los pasos 2 y 4. También se relacionan los pasos 1 y 6 a medida que llegan nuevos datos al sistema. 

Hasta el momento nos hemos centrado en las etapas de obtención, preprocesamiento y exploración de datos. El paso siguiente corresponde a un *refinamiento* de la información, con el fin de entrenar modelos de aprendizaje de alto rendimiento sobre los datos que se poseen. Este refinamiento ve su primera etapa en los procesos exploratorios (perfilamiento de datos y preprocesamiento inicial), para luego generar transformaciones y selecciones sobre las variables disponible. Dichas transformaciones y selecciones logran un mejor rendimiento en la medida que se obtienen por medio de *conocimiento del área* (*domain kwnlegde*), es decir, utilizando el conocimiento existente, con respecto al problema que se desea modelar. Sin embargo, existen ciertas técnicas *estándar* y algunas *automáticas* que permiten depurar los datos para obtener atributos y transformaciones de interés. 

En general, el proceso de refinamiento de datos se conoce como **ingeniería de características**, donde las características o *features* hacen referencia a los atributos y sus transformaciones. El proceso de ingeniería de características pasa generalmente por:

Explorar los datos y ajustar las variables disponibles, de manera tal, que se gane compatibilidad con las hipótesis de ciertos modelos de aprendizaje automático.

Posteriormente, se derivan características de interés, ya sea por medio de composición de atributos o transformaciones sobre estos. En esta parte, es una buena práctica, buscar transformaciones que hagan sentido con el problema modelado, esto no siempre se cumple (ej: una transformación polinomial sobre los datos no necesariamente es interpretable en ciertos problemas, pero puede aumentar considerablemente el rendimiento). 

Se continua, seleccionando caraterísticas, esto se puede hacer por medio de análisis estadísticos y algoritmos puntuación. Es necesario entender, que aunque en primera instancia más variables deberían llevarnos a mejores resultados, en la realidad, pueden aparecer caraterísticas que añadan ruido al sistema, ya sea por no estar relacionadas con el fenómeno estudiado o por presentarse problemas en su recolección. En tal contexto, cobra importancia la exlusión de dichas variables o de manera reciproca, la selección de aquellas que aseguren el mejor desempeño. 

Finalmente, se procede a entrenar y evaluar modelos sobre las métricas relevantes con el problema a trabajar, esta etapa se comunica constantemente con la de generación y selección de caraterísticas, pues las transformaciones que se decidan utilizar dependen en cierta medida de los modelos que se implementarán. 

A continuación se estudian ciertas técnicas de ingeniería de características estándar.

## Transformaciones de Atributos

Ya hemos estudiado técnicas de transformación por medio de preprocesamiento. A continuación se añaden y estudian nuevas metodologias además de aplicar algunas técnicas previas en la base de datos `HousePricing`. 

### Preprocesamiento inicial

Se cargan los datos y se agrupan sus columnas según tipo de dato

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

df = pd.read_csv('../S5/data/train.csv', index_col = 'Id')

Una de las técnicas más básicas de ingeniería de características es la **modificación de tipo de dato**, en este caso se transforman los datos reconocido como tipo `object` a tipo `str`.

In [ ]:
object_type_set = [col for col in df.columns if df[col].dtype == 'O']
df = df.astype({col:'str' for col in object_type_set})

Para trabajar de manera más sencilla con los datos se procede a agrupar los atributos por tipo de dato

In [ ]:
cat_cols = [
    'MSZoning', 'Street', 'Alley', 'LandContour', 'LotConfig', 'Neighborhood',
    'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle',
    'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'Foundation',
    'Heating', 'CentralAir', 'Electrical', 'GarageType', 'PavedDrive',
    'MiscFeature', 'SaleType', 'SaleCondition'
]

ordinal_cols = [
    'LotShape', 'Utilities', 'LandSlope', 'ExterQual', 'ExterCond', 'BsmtQual',
    'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'HeatingQC',
    'KitchenQual', 'Functional', 'FireplaceQu', 'GarageFinish', 'GarageQual',
    'GarageCond', 'PoolQC', 'Fence'
]


num_cols = [
    'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond',
    'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2',
    'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
    'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
    'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces',
    'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
    'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal',
    'MoSold', 'YrSold','SalePrice'
]

Se genera un *mapping* para obtener un datset multindexado

In [ ]:
mapping = [('numeric', col) for col in num_cols]
mapping.extend([('categorical', col) for col in cat_cols])
mapping.extend([('ordinal', col) for col in ordinal_cols])

df = df.reindex(columns=num_cols + cat_cols + ordinal_cols)
df.columns = pd.MultiIndex.from_tuples(mapping)

Obtenemos visualizaciones según tipo de variable

In [ ]:
def type_plotter(df, dtype, nrows=6):
    '''Permite graficar subconjuntos de variables en un dataframe multindexado
    
    Toma como argumento un dataframe de pandas df multi-indexado y un nivel 
    de multi - indice asociado a un subconjunto de columnas. Obtiene graficos
    univariados asociados a dicho subconjunto de varaibles.
    
    Args:
    -----
    
    df: DataFrame
        El conjunto de datos a explorar.
    
    dtype: String
        El nombre del nivel a visualizar
    
    nrwos: int
        La cantidad de filas a generar en la matriz de graficos
    
    Returns: 
    -------        
        None
        Se genera una visualizacion de matplotlib.
    '''

    cols = df[dtype].shape[1] // nrows
    resto = df[dtype].shape[1] % nrows

    fig, ax = plt.subplots(nrows=nrows + 1, ncols=cols, figsize=[17, 17])

    # Se remueve el resto de plots
    list(map(lambda a: a.remove(), ax[-1, -(cols - resto):]))

    fig.tight_layout()
    # Se define un titulo y su ubicacion
    fig.suptitle('Distribuciones Univariadas typo: ' + dtype,
                 fontsize=20,
                 x=0.5,
                 y=1.05)
    '''
    Se recorre cada axis, para cada columna del dataframe, se genera un grafico 
    distinto en funcion del tipo de dato.

    '''
    df_cols = df[dtype].columns
    for axis, col in zip(ax.flatten(), df_cols):
        try:
            # Graficos para datos numericos
            sns.distplot(df[(dtype, col)], ax=axis, rug=True)

        except RuntimeError:
            sns.distplot(df[(dtype, col)], ax=axis, rug=True, kde=False)

        except ValueError:
            sns.countplot(df[(dtype, col)], ax=axis)

        axis.set_xlabel(col, fontsize=15)

    # Se ajusta el espaciado interno entre subplots
    w, h = (.4, .4)
    plt.subplots_adjust(wspace=w, hspace=h)

Generamos visualizaciones para las variables numéricas

In [ ]:
type_plotter(df,'numeric')

El manejo de valores faltantes corresponde a una técnica de preprocesamiento ya estudiada y previa a las transformaciones de atributos buscadas en ingeniería de características. Por tal motivo, se hace un tratamiento rápido de los valores faltantes, sin ahondar en detalles. En este apartado, se reemplazan las variables con texto 'nan' por su objeto equivalente

In [ ]:
df.replace('nan',np.nan, inplace = True) 

Se observa la distribución porcentual de los valores perdidos

In [ ]:
def find_missing_per(df):
    na_percent = (df.isnull().sum()/len(df))[(df.isnull().sum()/len(df))>0].sort_values(ascending=False)

    return pd.DataFrame({'Missing Percentage':na_percent*100})

find_missing_per(df)

Se genera una función auxiliar para tratar con multi indices

In [ ]:
def indexer(cols, t_c = df.columns):
    '''Genera columnas multinivel a partir de nombres de columna planos.'''
    
    set_to_tuple = set(*[cols])

    tuples = [
        i for i in t_c if set_to_tuple.intersection(set(i))
    ]
    
    return tuples

Se llenan los valores faltantes

In [ ]:
# Se transforma nan -> None
to_none = [
    'FireplaceQu', 'Fence', 'Alley', 'MiscFeature', 'PoolQC', 'MSSubClass',
    'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'BsmtQual',
    'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'MasVnrType'
]

# Fill mapper
to_fill = {key: 'None' for key in indexer(to_none)}

# Se transforma nan -> 0
to_0 = [
    'GarageYrBlt', 'GarageArea', 'GarageCars', 'BsmtFinSF1', 'BsmtFinSF2',
    'BsmtUnfSF', 'TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath', 'MasVnrArea'
]
to_fill.update({key: 0 for key in indexer(to_0)})

# Se llenan los valores faltantes
df.fillna(to_fill, inplace=True)

Se estudia el dataset luego de llenar los valores anteriores

In [ ]:
find_missing_per(df)

En este caso se tiene que las variables `LotFrontage` y `Electrical` presentan valores faltantes. Para la variable `Electrical` esto representa una cantidad muy baja de observaciones, por lo que simplemente se pueden eliminar, mientras que para la variable `LotFrontage` se escoge llenar los valores faltantes por medio de una subagrupación en función de la variable 'Neighborhood', con esto, se asigna el valor de la mediana de 'LotFrontage' en para cada subgrupo. 

In [ ]:
group_median = lambda x: x.fillna(x.median())

df.dropna(axis=0, subset=indexer(['Electrical']), inplace= True)

df[indexer(['LotFrontage'])] = df.groupby(indexer(['Neighborhood']))[indexer(['LotFrontage'])].transform(
    group_median)

finalmente no se tienen valores faltantes

In [ ]:
len(find_missing_per(df))

El siguiente paso es trabajar con los valores anómalos. Nuevamente, este paso es previo a la transformación de atributos por lo que no se estudia en profundidad.

In [ ]:
varlist = [
    'GrLivArea', 'TotalBsmtSF', '1stFlrSF', 'MasVnrArea', 'GarageArea',
    'TotRmsAbvGrd'
]

from sklearn.preprocessing import RobustScaler
from sklearn.cluster import DBSCAN


def scatter_vs_price(df=df,
                     varlist=varlist,
                     nrows=3,
                     db_min_samples=5,
                     db_eps=0.99):
    
    '''Grafica una lista de variables contra SalePrice.
    
    Acepta como argumento una lista de variables, cada una de estas es 
    escalada de manera robusta, posteriormente se entrena una aglomeracion
    usando dbscan para resaltar los posible outliers.
    
    Parameters:
    -----------
    
    df: Pandas DataFrame
        El conjunto de datos de HousePricing
    
    varlist : List
              Lista de variables a estudiar
    
    nrows: int
           Cantidad de filas a graficar (argumento de subplots)
    
    db_min, db_eps: int, float
                    Hiperparametros de DBSCAN
    
    Returns:
    --------
        Visualizacion: None
                       Visualizacion de las variables estudiadas contra el precio
        
        Outliers: Dict
                  Diccionario de posibles outliers
    '''
    outliers = dict()
    
    ncols = len(varlist) // nrows
    resto = len(varlist) % nrows

    fig, ax = plt.subplots(nrows + 1, ncols, figsize=[13, 10])
    # Se borran las axis innecesarias
    for a in ax[-1, -(ncols - resto):]:
        a.remove()
    ax = ax.ravel()

    for a, var in zip(ax, varlist):
        
        X = df[indexer([var, 'SalePrice'])]
        scaler = RobustScaler()
        X = scaler.fit_transform(X)

        outlier_detection = DBSCAN(min_samples=db_min_samples, eps=db_eps)
        C = outlier_detection.fit_predict(X)

        a.scatter(x=X[:, 0], y=X[:, 1], c=C, alpha=0.5)

        #a.axvline(x=4600, color='r', linestyle='-')
        title = 'Scaled' + var + '- Price scatter plot'
        a.set_title(title, fontsize=15, weight='bold')
        outliers.update({var:C})
        
    fig.tight_layout()
    return outliers

Se estudian las variables seleccionadas

In [ ]:
outliers = scatter_vs_price()

Por simplicidad, se eliminan los valores anómalos.

In [ ]:
# Se recolectan los indices de valores anomalos
outlier_idx = pd.Index({})
for key, val in outliers.items():

    outlier_locs = [*map(bool, val)]
    columns = indexer([key])

    outlier_idx = outlier_idx.union(df.loc[outlier_locs, columns].index)
    
# Se eliminan las filas con valores anomalos
df.drop(index=outlier_idx, axis=0, inplace=True)

Si bien, luego de este preprocesamiento se puede trabajar en transformaciones sobre los datos, se debe recordar, que las etapas de preprocesamiento y manejo de caraterísticas se comunican constantemente, generando ciclos en el flujo de trabajo. 

### Manejo de Carácteristicas Básico


A continuación se procede a producir características relevantes, una primera aproximación a este procedimiento es comenzar por introducir transformaciones sobre las variables y modelar la interacción entre estas. En este apartado, es necesario discutir el término de **Data Leakage**. 

**Data Leakage** hace referencia a la *fuga* de información entre conjuntos destinados para entrenamiento y test. Si se produce esta fuga, significa que se esta utilizando información no valida en procesos de entrenamiento, lo cual genera modelos sobre optimistas en sus métricas de evaluación. En la práctica, un modelo con fuga de información puede ser incluso inútil en producción. Para evitar este fenómeno, basta realizar las transformaciones de datos pertinentes dentro de los procesos de validación y entrenamiento. Otra buena práctica en este aspecto, es generar un conjunto de validación adicional para comprobar métricas de rendimiento luego de realizar selección de modelo.

A modo de ejemplo: si en un problema de modelamiento se normalizan ciertas variables utilizando la información total del dataset y luego se procede a generar particiones de entrenamiento y validación, entonces los elementos ubicados en los conjuntos de validación habrán aportado información al proceso de normalización pues influyeron en los calculos de media y desviación estándar. Esto puede mejorar de cierta manera el rendimiento en predicción, dando como resultado un modelo sesgado.

Lo primero que se hará es generar un conjunto de validación utilizando el 10% de los datos, este conjunto se utiliza al final del proceso de selección de modelos para verificar la capacidad de generalización.

In [ ]:
from sklearn.model_selection import train_test_split

df_train, df_val = train_test_split(df, test_size = .1)

A continuación, se procede a estudiar ciertos atributos del conjunto de datos

In [ ]:
f, ax = plt.subplots(figsize=(8, 7))

sns.set_style("white")
sns.set_color_codes(palette='deep')


sns.distplot(df_train['numeric']['SalePrice'], color="b");
ax.xaxis.grid(False)
ax.set(ylabel="Frecuancia")
ax.set(xlabel="SalePrice")
ax.set(title="Distribucion de SalePrice")

sns.despine(trim=True, left=True)
plt.show()

Vemos que tiene similitud a una distribución normal, para ello se hace un test de normalidad

In [ ]:
from scipy import stats

def norm_test(data = df_train['numeric']['SalePrice'] ):

    k, p = stats.normaltest(data)

    alpha = 1e-3
    print("p =",p)

    if p < alpha: 
        print("Se puede rechazar la hipotesis nula")
    else:
        print("No se puede rechazar la hipotesis nula")

In [ ]:
norm_test()

Luego la variable `SalePrice` no se comporta como una normal. En estos casos. Una técnica de manejo de características básica es la transformación de Box-Cox (ya vista), Podemos transformar esta variable en función de tal método

In [ ]:
from sklearn.preprocessing import PowerTransformer

# Se utiliza una funcion auxiliar para transformar la serie
to_array = lambda pd_series: pd_series.values.reshape(-1, 1)
y = to_array(df_train['numeric']['SalePrice'])

transformer_bc = PowerTransformer(method='box-cox')

y = transformer_bc.fit_transform(y)

# Se aplica el test de normalidad a la variable de interes
norm_test(y)

Luego la transformación de Box - Cox permite normalizar de manera sencilla la distribución de la variable `SalePrice`. En general, todas las técnicas de preprocesamiento vistas anteriormente (escalar, normalizar,códificación dummy, etc...) entran en el manejo de básico de características. 

**Ejercicio**

1. Visualice las variables numéricas, calcule su asimetría estadística (skewness) por medio del método `.skew()`. Defina un valor umbral (ej: 0.6). Luego aplique una transformación de potencia sobre las variables más asimétricas. 

**Obs**: Lo anterior permite normalizar de manera más robusta aquellas variables que por naturaleza se alejan de ser normales.

Otra manera de generar características de interés es por medio de interacciones. En este caso se hace uso del conocimiento del área que se trabaja. 

Por ejemplo, las variables `TotalBsmtSF`, `1stFlrSF` y `2ndFlrSF` representan unidades de área en pies cuadrados para el subterraneo, primer y segundo piso. Con esta información se puede crear la variable `TotalSF` que representa la superficie total de una vivienda

In [ ]:
df_train.loc[:, ('numeric','TotalSF')] = df_train['numeric']['TotalBsmtSF'] + \
                                         df_train['numeric']['1stFlrSF'] + \
                                         df_train['numeric']['2ndFlrSF']

Se puede también general la variable `YearsSinceRem` que permite codificar la cantidad de años que han pasado desde la remodelación de una vivienda

In [ ]:
df_train.loc[:,('numeric','YearsSinceRemodel')] = df_train.loc[:,('numeric','YrSold')]\
                                          - df_train.loc[:,('numeric','YearRemodAdd')]

De la misma manera, se puede agregar la variable `TotalQual` que representa el puntaje total de calidad asociado a una vivienda

In [ ]:
df_train.loc[:,('numeric','TotalQual')]  = df_train.loc[:,('numeric','OverallQual')]+\
                                           df_train.loc[:,('numeric','OverallCond')]

En general, se pueden aplicar transformaciones exponenciales (como en el caso de Box-Cox), lineales (como los dos casos anteriores) o de cualquier otro tipo (trigonometricas, polinomiales) en función del tipo de problema que se trabaje, siempre y cuando la transformación tenga un trasfondo en el fenómeno.

In [ ]:
'''
Para recuperar multi indexado: 
df_train = pd.read_csv('df_train_v1.csv', header=[0,1], index_col=0)
'''

df_train.to_csv('df_train_v1.csv')

**Ejercicio**

Otra forma de inducir interacciones es por medio de variables dummy, en este caso se procede a generar codificaciones one hot. En este contexto, si se posee una variable codificada de esta forma y se multiplica por otra, ocurre que la nueva variable generada modela la interesección de eventos entre esas variables. 

1. Visualice las variables categóricas, genere un esquema de codificación y modele interacciones entre variables. Cuantifique estadísticamente el efecto de las variables de interacción con la variable de respuesta. 

## Selección de características 

En conjunción con el estudio, perfilamiento y aplicación de transformaciones iniciales a los datos, se hace necesario utilizar técnicas de selección de característcas. Estas permiten mejorar métricas de rendimiento en predicción. 

Las técnicas de selección de característcas se pueden clasificar como **no supervisadas** que no usan la variable de respuesta (análisis de correlaciones en contextos de multicolinealidad). Otra categroría son las técnicas **supervisadas** , estas hacen uso de la variable de respuesta, y pueden separarse en técnicas *envolventes* o *wrappers*, las cuales seleccionan variables en función de los resultados provistos por algoritmos de aprendizaje. Se observan también los métodos de *filtrado*, que seleccionan subconjuntos de variables en función de ciertos estadísticos, valores umbrales o scoring  (pueden ser no supervisados). Además, se encuentran los métodos de selección supervisada *implícitos*, estos corresponden a metodologías naturales dentro de ciertos algoritmos de aprendizaje (ej: selección de variables por medio de regresión LASSO). Por último se pueden utilizar técnicas de **reducción de dimensionalidad** que permiten proyectar los datos input a dimensiones más bajas. 

Sci-kit Learn provee de la API `sklearn.feature_selection ` que proporciona métodos de ayuda en este apartado.

A continuación se estudian métodos de **selección de caraterísticas univariadas**.

### Umbral de varianza 

Este método permite seleccionar variables por *filtrado* no supervisado y es un acercamiento simple que se basa en la premisa de que aquellas características con muy poca varianza no aportan información al modelo. 

**Ejemplo**

Se observan las características ordinales del conjunto de entrenamiento


In [ ]:
type_plotter(df_train,dtype='ordinal')

En este caso se observa que las variables `Functional` y `PoolQc` poseen una baja varianza (entre otras variables). Se importa el método de seleccion de caraterísticas por medio de umbral de varianza y se estudia su efecto en el conjunto de entrenamiento, para ello, es necesario codificar las variables estudiadas.

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder

Se generan pipelines sobre las variables ordinales

In [ ]:
# Adquieren las categorias de cada variable
ordinals = df_train['ordinal']

ordinal_cat = [[*ordinals[col].unique()] for col in ordinals.columns]

ord_pipe = Pipeline(
    steps=[('ordinal', OrdinalEncoder(categories = ordinal_cat))])

In [ ]:
X_train = df_train.drop(('numeric','SalePrice'), axis=1).dropna(how = 'all').copy()

# Variable dependiente
y_train = df_train['numeric']['SalePrice'].copy()

# Se preparan los datos
X_prep = ord_pipe.fit_transform(X_train['ordinal'])

In [ ]:
X_train_enc = pd.DataFrame(X_prep, columns=X_train['ordinal'].columns).astype('int')

Las varianzas asociadas al conjunto de características anterior viene dado por:

In [ ]:
X_train_enc.var()

Se importa el método de umbral de varianza y se aplica a las características del conjunto de datos

In [ ]:
from sklearn.feature_selection import VarianceThreshold

sel = VarianceThreshold(threshold=.47)
X_filtered = sel.fit_transform(X_train_enc)

In [ ]:
X_train_enc.shape

In [ ]:
X_filtered.shape

Con lo que se filtran 9 caracteristicas, para obtener las caraterísticas seleccionadas se hace uso del método `.get_support()` para obtener una mascara de los indices seleccionados

In [ ]:
X_train['ordinal'].columns[sel.get_support()]

### Métodos Univariados para Regresión 

Dentro de las técnicas clásicas de selección de variables en contextos de **regresión** encontramos los métodos:

**Input numérico u Ordinal**

* Coeficiente de correlación de Pearson (scoring lineal)

Este coeficiente mide la relación lineal entre dos variables aleatorias gaussinas y ha sido utilizado de manera frecuente durante el curso. Para seleccionar características utilizando este coeficiente se puede hacer uso de la función `f_regression` del módulo `feature_selection` de Scikit-learn.


* Coeficiente de Spearman (scoring no lineal) 

El coeficiente de correlación de Spearman corresponde a una medida de correlación de *rango*, esto se refiere a que permite cuantificar relaciones entre variables ordinales. En este caso, si se trabaja con variables numéricas, es necesario categorizarlas en intervalos ordenados para luego compararlas. La ventaja de esto es que no se requiere especificar distribuciones para los datos, por lo que es un test no paramétrico. Este coeficiente permite describir si la relación entre dos variables aleatorias es *monotona*.

Si se poseen $N$ $X_i$ e $Y_i$ de las variables aleatorias $X$ e $Y$, es posible calcular el coeficiente de correlación de Spearman $r_s$ generando transformaciones de rango $\pi_X$ y $\pi_Y$ sobre los valores $X_i$ e $Y_i$. Lo que esta transformación hace es ordenar las muestras asociadas a cada variable aleatoria asociandoles una posición.

**Ejemplo**

Se define una variable aleatoria y se calcula su transformación de rango.

In [ ]:
X = np.random.rand(10)

def pi_x(X):
    X_ = X.copy()
    X_.sort()
    return [*enumerate(X_,start=1)]

In [ ]:
pi_x(X)

Cuando dos o más valores se repiten, se debe definir una regla de desempate, esta consiste en asignar rangos promedio, es decir, si el valor $X_i$ se repite $l$ veces, entonces se les asigna el promedio de sus rangos, es decir $\pi_X(X_i^{(k)}) = \frac{1}{l}\sum_{j=1}^l \pi_X(X_i^{(j)})$ para todo $k = 1,...,l$ donde $X_i^{(k)}$ representa cada copia de $X_i$.

**Ejercicio**

1. Modifique la función de asignación de rangos para que implemente esta regla de desempate.

Haciendo uso de las transformaciones $\pi_X$ y $\pi_Y$ se calcula 
$r_s$ por medio de:
$$
r_{s}=\frac{\operatorname{cov}\left(\pi_{X}, \pi_{Y}\right)}{\sigma_{\pi_{X}} \sigma_{\pi_{Y}}}
$$
Donde $\sigma_{\pi_{X}}$ y $\sigma_{\pi_{Y}}$ corresponden a las desviaciones estándar sobre los rangos de las variables. El calculo anterior corresponde por tanto al coeficiente de correlación de Pearson sobre la categorización ordinal de las variables de interés. Para calcular el coeficiente de correlación de Spearman entre las variables de numéricas se puede hacer uso del método de pandas `.corr()`.

**Ejemplo**

Se calcula el coeficiente de correlación de Spearman sobre las variables numéricas

In [ ]:
spearman = df_train['numeric'].corr(method='spearman')

In [ ]:
spearman.nlargest(5,columns='SalePrice')['SalePrice']

In [ ]:
spearman.nsmallest(5,columns='SalePrice')['SalePrice']

un coeficiente de Spearman muy cercano a 1 o a -1 indica relaciones monotonas entre las variables, por otra parte un coeficiente cercano a 0 sugiere ausencia de tal tipo de relaciones. Con esto es posible hacer un filtrado de variables poniendo un umbral de correlación.

**Ejercicio**

1. La función `spearmanr` de `scipy.stats` permite tener acceso a un valor de significancia para contrarrestar la hipotesis nula _" No hay relación entre las caracteristicas"_. Seleccione ciertas variables numéricas y utilizando dicha función.

* Coeficiente de correlación de Kendall (scoring no lineal) 

Este coeficiente mide la *concordancia* entre los rangos  de dos variables aleatorias . Los rangos asociados a cada valor son calculados de la misma manera que para el coeficiente de correlación de Spearman. Si se obtienen dichos rangos, es posible calcular el *número de pares concordantes* $C$ y *número de pares no concordantes* $D$, para ello, si se poseen $N$ muestras $X_i$ e $Y_i$ de las variables aleatorias $X$ e $Y$, se generan sus transformaciones de rango correspondientes $x_i = \pi_X(X_i)$ y $y_i = \pi_Y(Y_i)$, luego se generan pares ordenados $(x_i, y_i)$. Para finalizar, el coeficiente de correlación $\tau$ de Kendall se obtiene mediante la formula:
$$
\tau=\frac{2}{n(n-1)} \sum_{i<j} \operatorname{sgn}\left(x_{i}-x_{j}\right) \operatorname{sgn}\left(y_{i}-y_{j}\right)
$$

**Ejemplo**

Se utiliza el coeficiente de correlación de Kendall con pandas.

In [ ]:
kendall = df_train['numeric'].corr(method='kendall')
kendall.nsmallest(5,columns='SalePrice')['SalePrice']

In [ ]:
kendall.nlargest(5,columns='SalePrice')['SalePrice']

En este caso, un valor cercano a 1 indica alta concordancia, cercano a -1 indica alta discordancia. Un valor cercano a 0 indica la ausencia de relación. 

**Ejercicios**

1. Utilice la función `kendalltau` del módulo `scipy.stats` para realizar pruebas de hipótesis sobre la relación entre variables numéricas y la variable predictiva.

2. Scikit-learn ofrece la función `f_regression` del módulo `feature_selection`. Esta permite realizar un test F univariado. Con este test se captura la dependencia lineal entre dos vectores. ¿Es posible interpretar el estadístico F como un puntaje de selección de variables? 

* Información Mutua

La información mutua entre dos variables $X$ e $Y$ busca medir dependecia. Así, si ambas son independientes, no existe información de $Y$ que se pueda obtener por medio de $X$ (lo mismo en el sentido contrario) luego el valor asociado de información mutua para estas variables será 0. Si por otra parte $X$ es una función determinista de $Y$ se tendrá un valor de información mutua máximo, en general, mayores valores de este valor indican mayor dependencia (puede ser usado como un scoring). En términos matemáticos, si $(X,Y)$ es un par de variables aleatorias, si su distribución conjunta se denota por $P_{(X,Y)}$ y sus marginales por $P_X$ y $P_Y$ se define la información mutua $I(\cdot,\cdot)$ como:

$$
I(X;Y) = D_{KL}(P_{(X,Y)} || P_X \otimes P_Y))
$$

Donde $D_{KL}$ es la divergencia de Kullback-Leibler. Esta se define para dos medidas de probabilidad $P, Q$ sobre $\mathcal{X}$, con $P$ absolutamente continua con respecto a $Q$ según la ecuación:
$$
D_{\mathrm{KL}}(P \| Q)=\int_{\mathcal{X}} \log \left(\frac{d P}{d Q}\right) d P
$$

En el caso discreto esto equivale a
$$
D_{\mathrm{KL}}(P \| Q)=\sum_{x \in \mathcal{X}} P(x) \log \left(\frac{P(x)}{Q(x)}\right)
$$
Con lo que la información mutua se puede escribir como:

$$
\mathrm{I}(X ; Y)=\sum_{y \in \mathcal{Y}} \sum_{x \in \mathcal{X}} P_{(X, Y)}(x, y) \log \left(\frac{P_{(X, Y)}(x, y)}{P_{X}(x) P_{Y}(y)}\right)
$$

La ventaja de esta medida de dependencia recae en que puede cuantificar relaciones de dependencia no lineales.

**Ejemplo**

Se aplica el criterio de información mutua para filtrar características, para ello se importa el módulo correspondiente

In [ ]:
from sklearn.feature_selection import mutual_info_regression

Y = df_train['numeric'].SalePrice.values.reshape([-1,])
X = df_train['numeric'].GrLivArea.values.reshape([-1,])

I = mutual_info_regression(df_train['numeric'].drop('SalePrice', axis=1),Y)
I

Se puede ver que hay variables indicadas como independientes de `SalePrice` por lo que puede ser beficioso para el modelo extraerlas.

**Ejercicios**

1. Genere un conjunto de datos `D` de 3 dimensiones y 1000 registros utilizando `np.random.rand`.

2. Genere una variable `y` dependiente de `D`, para ello, `y` debe depender de manera lineal de la primera columna de `D` y de manera no lineal de la segunda columna de `D` (use una transformación trigonométrica por ejemplo). Añada ruido aleatorio normal. 

3. Haga un test F para medir la relación entre las columnas de `D` (características) e `y`. 

4. Haga calcule la información mutual entre las características de `D` e `y`. Compare con los resultados de del test F. 

### Métodos Univariados para Clasificación

Cuando la variable de respuesta es discreta se pueden aplicar las siguientes técnicas.

**Input Númerico**

* Test F ANOVA 

Este método ya fue estudiado. Permite detectar si la variable de respuesta (discreta) genera grupos significativamente diferenciables en la variable numérica input. Scikit-learn ofrece la función `f_classif` del módulo `feature_selection` para realizar la versión one-way de este test. Observe que se puede utilizar para inputs numéricos en problemas de regresión. Este test requiere que los grupos inducidos por las categorías sean distribuidos de manera normal, lo cual dificilmente se cumple en la práctica.

* Información Mutua

Este método es aplicable a problemas de clasificación con inputs numéricos. 

**Inputs discretos**

* Test Chi cuadrado

El test de independencia $\chi^2$ permite determinar si existen relaciones de dependencia significativas entre dos variables discretas. El estadístico $\chi^2$ comprueba la existencia de diferencias significativas entre las frecuencias observadas y las frecuencias esperadas para las variables estudiadas. Acá la hipótesis nula es `No exsite una asociación entre las variables`. Para la construcción de este test se genera una tabla de contingencia, esta consiste en una matriz con entradas del tipo $O_{ij}$, cada una de las cuales representa la cantidad de veces que se observa la categoría (feature) $x_i$ versus la categoría $y_j$ (respuesta). De esta forma, se pueden obtener totales por fila $N_{i,.}$ y por columna $N_{.,j}$. Con estos valores se obtienen los *conteos esperados* $\hat{E}_{ij}$ por medio de:
$$
\hat{E}_{i, j}=\frac{\left(N_{i, .}\right)\left(N_{., j}\right)}{\sum_{i}N_{i,.} + \sum_{j}N_{.,j}}
$$

Así, el estadístico asociado a la relación de categorías $i,j$ viene dado por:

$$
\chi^{2}=\sum_{i, j} \frac{\left(O_{i j}-\hat{E}_{i, j}\right)^{2}}{\hat{E}_{i, j}}
$$

Tal valor permite hacer pruebas de hipótesis o ser considerado como un puntaje para selección de características.

**Ejemplo**

Se aplica el test chi cuadrado sobre las variables categóricas, para ello, se transforma la variable `SalePrice` a una variable ordinal y se procesa por medio de encoders de Scikit-learn

In [ ]:
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

# Features
X_train = df_train['categorical'].copy()
X_train.dropna(how='all', inplace=True)

# Variable dependiente
y_train = df_train['numeric']['SalePrice'].copy()
y_train.dropna(how='all', inplace=True)
y_train = pd.cut(y_train, bins=5)
y_train = np.array(y_train).reshape([-1,1])

# Se preparan los datos
encoder_one_hot = OneHotEncoder(sparse=False)
encoder_ordinal = OrdinalEncoder()

X_cat_enc = encoder_one_hot.fit_transform(X_train)

X_cat_enc = pd.DataFrame(
    X_cat_enc, columns=encoder_one_hot.get_feature_names()).astype(int)

y_train_enc = encoder_ordinal.fit_transform(y_train)

Finalmente se obtiene un arreglo de puntajes y valores de significancia para cada categoría

In [ ]:
from sklearn.feature_selection import chi2

chi2(X_cat_enc,y_train_enc)

**Ejercicio**

1. El método de información mutua funciona para variables discretas en problemas de clasificación. Investigue como implementarlo en selección de características en tal contexto.

En general, los métodos univariados de selección de caractrísticas permiten realizar selecciones basándose en valores interpretables como *puntajes*. Si se ve este proceso de selección como un filtrado de caracteŕisticas, entonces es sencillo implementarlo dentro de una *pipeline* de preprocesamiento, para esto, Scikit-learn ofrece los objetos `SelectKBest` para filtrar características según un número de dimensiones objetivo, `SelectPercentile` que permite remover variables según un porcentaje objetivo de dimensiones.

**Ejemplo**

Se seleccionan 5 variables categóricas según el resultado del test chi cuadrado.


In [ ]:
from sklearn.feature_selection import SelectKBest

# Se entrena y transfroma el selector
selector = SelectKBest(
    k=5,
    score_func=chi2,
)
selected_data = selector.fit_transform(X_cat_enc, y_train_enc)

# Se obtienen las categorias
selected_cats = selector.get_support()
selected_cats = X_cat_enc.columns[selected_cats]

print(
    'Categorias seleccioandas: \n',
    *[str(i + 1) + '.- ' + str(h) + '\n' for i, h in enumerate(selected_cats)])

**Ejercicios**

1. Utilice `SelectPercentile` del módulo `feature_selection` para seleccionar el 10% de las categrías con mayor puntuación según el estadístico chi cuadrado.

2. Lo métodos de proporcion de falso positivo (FPR), proporción de descubrimientos falso (FDR) y error por familias (FWE) permiten seleccionar caracteríssticas utilizando un valor de significacia versus p-valores en tests estdísticos. Investigue su formulación y aplique selección de variables utilizando los métodos `SelectFpr`, `SelectFdr` y `SelectFwe` respectivamente.

### Interludio: Métodos de Estimación basados en Árboles

Los métodos basados en arboles operan particionando el espacio de características en un conjunto de rectangulos, luego se ajusta un modelo simple (cómo una constante $c$) en cada uno de estos rectangulos, con el fin de predecir la respuesta de los datos agrupados en esta región del espacio.

En concreto este proceso se reduce a encontrar cuales son las variables que mejor segmentan el conjunto de datos (con una desigualdad simple), en el caso binario para la primera etapa del algoritmo el conjunto de entrenamiento original es particionado en 2 subconjuntos. Este proceso se repite de forma recursiva en cada una de las separaciones. De esta forma se segmenta el conjunto de entrenamiento quedando asociada cada muestra a un nodo terminal del árbol ensamblado.

Después del proceso de crecimiento del arbol recién descrito dependiendo del contexto del problema abordado, puede seguir una etapa de podado del arbol, lo cual permite manejar el tamaño y problemas con **sobre-ajuste** en el conjunto de entrenamiento.

#### CART: Arboles de Regresión y Clasificación

En primer lugar se aborda la construcción para el problema de regresión, luego se revisará un método para construir clasificadores modificando algunos aspectos de la construcción que antecede.

##### Regresión:

Sea un conjunto de datos $\{ (x_i,y_i) \}_{i=1}^N$, donde cada $x_i \in \mathbb{R}^p$ e $y_i \in \mathbb{R}$ corresponde a la respuesta. Se define $X = (x_1,\ldots, x_N)^T \in \mathbb{R}^{N \times p}$. Esto corresponde al escenario clásico que se da en un problema de regresión.

La idea del algoritmo es generar reglas de desición automáticas para segmentar los puntos del conjuntos de entrenamiento y la topología de las regiones de desición. Para este desarrollo usaremos un esquema de partición binario, es decir el árbol resultante será un árbol binario.

Supongamos que ya se tiene una partición con $M$ elementos $R_1, \ldots, R_M$; donde se modela una respuesta constante $c_m$ en cada región, es decir:

\begin{equation}
f(x) = \sum_{m=1}^M c_m I{(x \in R_m)}
\end{equation}

Donde $I$ corresponde a la función indicatríz.

Al adoptar como criterio de perdida la minimización de la suma de los errores cuadráticos ($\sum(y_i - f(x_i))^2)$ es simple demostrar que el $c_m$ óptimo en cada región $R_m$ corresponde al promedio entre las respuestas, es decir:

\begin{equation}
\hat{c}_m = \frac{1}{|R_m|}\sum_{i:x_i \in R_m} y_i
\end{equation}

Encontrar la mejor partición en término de los errores cuadráticos es computacionalmente costoso, incluso prohibitivo. Por esta razón se emplea un esquema de optimización **glotón**, partiendo con todos los datos, considerando una variable $j$ para partición y un punto $s$, se difinen los siguiente semi planos:

\begin{align}
R_1(j,s) &= \{ x \in X | x_j \geq s\} \\
R_2(j,s) &= \{ x \in X | x_j < s\}
\end{align}

Luego para encontrar las variables $j,s$, se resuelve el siguiente problema:

\begin{equation}
\min_{j,s} \left \{ \min_{c_1} \sum_{i:x_i \in R_1(j,s)} (y_i-c_1)^2 + \min_{c_2} \sum_{i:x_i \in R_2(j,s)} (y_i-c_2)^2 \right \}
\end{equation}

para cualquier $j,s$ la elección óptima de $\hat{c}_1$ y $\hat{c}_2$ corresponderá al promedio los $y_i$ en las regiones $R_1$ y $R_2$ respctivamente.

Considerando que para cada $j$ variable de segmentación es facil determinar el punto $s$ óptimo. Es factible recorrer todas las características para encontrar el par óptimo $(j,s)$.

Habiendo obtenido el primer corte de los datos $R_1$ y $R_2$, el proceso de segmentación se realiza de forma recursiva en cada una de las regiones encontradas de forma sucesiva.

La pregunta que aparece de forma natural en esta etapa del proceso es que tan profundo debe ser extendido el arbol. Claramente una arbol demasiado grande puede conllevar a un **sobre-ajuste** del conjunto de entrenamiento, generando malas propiedades de generalización del modelo.

La estrategía que suele ser empleada es definir un parámetro $n$ que representa el tamaño mínimo que puede tener una hoja del arbol. De esta forma el arbol crece hasta que cada hoja de este tiene un tamaño a lo más $n$. Este arbol producto del proceso de separar sucesivamente los datos se define cómo $T_0$.

Para evitar lo mencionado en el parrafo anterior despues de crecer el arbol, se lleva a cabo un proceso de poda, es decir colapsar nodos del arbol (no terminales, hojas). Esto será explicado y se realiza en base a un criterio que considera la función de costo y la complejidad del arbol (tamaño), un criterio de **costo-complejidad**.

Se define el sub arbol $T \subset T_0$ cómo cualquier arbol que puede ser obtenido a partir de $T_0$ colapsando cualquier cantidad de nodos internos (no terminales) del arbol. Se indexan los nodos terminales de $T$ por $m$, cada uno representando una región $R_m$, adicionalmente $|T|$ representa la cantidad de nodos terminales de $T$.

Se define:

\begin{align}
N_m =& |R_m|\\
\hat{x}_m =& \frac{1}{N_m} \sum_{i:x_i \in R_m} y_i \\
Q_m(T) =& \frac{1}{N_m} \sum_{i:x_i \in R_m} (y_i - \hat{c}_m)^2
\end{align}

Con esto se define el criterio de **costo-complejidad** cómo:

\begin{equation}
C_{\alpha}(T) = \sum_{m=1}^{|T|} N_m Q_m(T) + \alpha |T| 
\end{equation}

La idea es encontrar para cada $\alpha$ el sub arbol $T_\alpha \subset T_0$ que minimice $C_\alpha(T)$. Se observa que $\alpha \geq 0$ funciona cómo un termino de penalización sobre el tamaño del arbol, valores grandes de $\alpha$ inducen arboles más pequeños. Si $\alpha= 0$ entonces se recupera $T_0$.

Para cada $\alpha$ se puede demostrar que existe un único $T_\alpha$ de tamaño mínimo que minimiza $C_\alpha$. Para encontrar $T_\alpha$ se usa la estrategía de podar el nodo que produce el menor incremento posible en $\sum_{m=1}^{|T|} N_m Q_m(T)$ hasta llegar a la raíz. Esto genera una secuencia finita de sub arboles, es posible demostrar que esta secuencia contine necesariamente a $T_\alpha$.

Para estimar el valor de $\alpha$ es posible recurrir a un esquema de validación cruzada (se recomienda 5-10) sobre un conjunto de validación. Finalmente se elige $\hat{\alpha}$ que minimíce la suma de errores cuadráticos en el esquema de validación cruzada. Finalmente el arbol óptimo será $T_\hat{\alpha}$.

Las **ventajas** de los este método se consisten en su capacidad para manejar variables numéricas y categóricas de manera nativa. Además requieren de un procesamiento casi nulo de los datos, son además modelos no paramétricos y se comporta bien en altas dimensiones. Por otra parte, las **desventajas** o limitaciones de esta familia de modelos recae en su alta propensión al sobreajuste y alta fragilidad en cuanto a cambios en el conjunto de datos. Por lo general, al trabajar con clases no balanceadas se producen problemas de sesgo de manera muy notoria.

**Ejercicio**

1. Observando de la formulación del modelo, ¿Qué significa que una variable sea seleccionada al comienzo para particionar? investigue sobre el atributo `feature_importances_` que poseen los modelos basados en arboles de Scikit-learn.

**Ejemplo**

Se genera un modelo basado en arboles de para estimar el precio en la base `HousePricing` en función de ciertas variables.

In [ ]:
interest = [
    'SalePrice', 'OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF',
    'FullBath', 'YearBuilt'
]

interest = indexer(interest)

se importa la clase `DecisionTreeRegressor` para hacer regresión sobre el precio de venta. Se genera además un conjunto de entrenamiento en con las variables de interés

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

def rmse(y,y_pred):
    return np.sqrt(mean_squared_error(y,y_pred))
    
tree_reg = DecisionTreeRegressor()

# Conjuto de entrenamiento
df_train_tree = df_train[interest].copy()

se entrena un modelo sobre los datos sin optimizar hiperparámetros

In [ ]:
# Separacion de datos en train y test usando pandas
X_train_tree = df_train_tree[interest[:-1]].sample(frac=.7)
Y_train_tree = df_train_tree.loc[X_train_tree.index,interest[-1]]

test_idx = df_train_tree.index.difference(X_train_tree.index)

X_test_tree = df_train_tree.loc[test_idx, interest[:-1]]
Y_test_tree = df_train_tree.loc[test_idx,interest[-1]]

# Se entrena el modelo
tree_reg.fit(X_train_tree, Y_train_tree)

# Se obtienen metricas de rendimiento en train

train_res = { 'R2': tree_reg.score(X_train_tree,Y_train_tree), 
              'rmse':  rmse(tree_reg.predict(X_train_tree),Y_train_tree)}
train_res

se obtienen el métricas de rendimiento sobre test

In [ ]:
test_res = { 'R2': tree_reg.score(X_test_tree,Y_test_tree), 
            'rmse':  rmse(tree_reg.predict(X_test_tree),Y_test_tree)}
test_res

si bien los resultados en entrenamiento parecen prometedores, en test se observa un baja capacidad de generalización, lo cual es una clara señal de sobreajuste, como era de esperar en este tipo de modelos. Para mejorar la capacidad de generalización haremos uso de un esquema de búsqueda por grilla según validación cruzada, para esto se importa el módulo `model_selection` especificando la función `GridSearchCV`

In [ ]:
from sklearn.model_selection import GridSearchCV

esta función toma como argumento un estimador, una grilla de hiperparámetros especificada como un diccionario, además se puede especificar la cantidad de conjuntos train / test que se quiere generar por medio de `cv` y la cantidad de procesos en paralelo que se realizan `n_jobs` entre otros parámetros. Se aplica el esquema de grilla sobre un regresor basado en arboles

In [ ]:
tree_reg = DecisionTreeRegressor(random_state=6202)

grid = {
    'max_depth': range(2, 11),
    'ccp_alpha': np.power(2, np.linspace(-5,7,num = 12)),
    'min_samples_split': range(2, 6),
    'min_samples_leaf': range(2, 6)
}

cv = GridSearchCV(tree_reg, param_grid=grid, cv=5, n_jobs=-1)

cv.fit(X_train_tree, Y_train_tree)

tree_reg_best = cv.best_estimator_
tree_reg_best

Se almacena el modelo obtenido en un formato accesible, esto se denomina **persistencia de modelos**

In [ ]:
import pickle

with open('tree_model.pkl','bw') as handler:
    pickle.dump(tree_reg_best,handler)

se recolectan las métricas de rendimiento correspondientes

In [ ]:
train_res_gs = { 'R2': tree_reg_best.score(X_train_tree, Y_train_tree), 
               'rmse': rmse(tree_reg_best.predict(X_train_tree),Y_train_tree)}

print('Resultados train: \n', train_res_gs , '\n')

test_res_gs = { 'R2': tree_reg_best.score(X_test_tree,Y_test_tree), 
              'rmse': rmse(tree_reg_best.predict(X_test_tree),Y_test_tree)}

print('Resultados test: \n', test_res_gs)

con lo que se observa cierto sobreajuste pero a la vez una mejora en la capacidad de generalización.

**Ejercicio**

1. ¿Cómo se producen las particiones train / test en el esquema automatizado por `GridSearchCV` para `cv = 5`?

#### Clasificación

El esquema es homólogo al de regresión, solamente se deben ajustar los criterios de separación de nodos y el podado de estos.

Cómo se trata de un problema de clasificación, los valores de la respuesta $y_i$ toma valores en un conjuntos discretos $1,\ldots, K$.

En el problema anterior se tomó como criterio de **pureza** dentro de los nodos la suma de errores cuadráticos, esto no es apropiado para clasificación.

En un nodo $m$ del árbol, que representa la región $R_m$ con $N_m$ observaciones, se define:

\begin{equation}
\hat{p}_{mk} = \frac{1}{N_m} \sum_{i: x_i \in R_m} I(y_i = k)
\end{equation}

como la proporción de observaciones de la clase $k$ en el nodo $m$. Se clasifican los puntos en el nodo $m$ según:

\begin{equation}
k(m) = \arg \max_k \hat{p}_{mk}
\end{equation}

es decir, se asignan los puntos en el nodo $m$ a la clase que mayor presencia tenga en el nodo.

Para el problema de clasificación se suele usar distintas forma de $Q_m(T)$ (pureza del nodo), los más comones son:

- **Error de clasificación:**
\begin{align}
Q_m^1(T) =& \frac{1}{N_m} \sum{i \in R_m} I(y_i \neq k(m))\\
=& 1- \hat{p}_{mk(m)}
\end{align}

- **Coeficiente de Gini:**
\begin{align}
Q_m^2(T) =& \sum_{k \neq k'} \hat{p}_{mk} \hat{p}_{mk'}\\
=& \sum_{k=1}^K \hat{p}_{mk}(1 - \hat{p}_{mk})\\
\end{align}

- **Entropía:**
\begin{equation}
Q_m^3(T) = -\sum_{k=1}^K \hat{p}_{mk}\log\hat{p}_{mk}
\end{equation}

Las ventajas de $Q_m^2$ y $Q_m^3$ sobre $Q_m^1$, es que estos son diferenciables, lo cual puede ser fundamental para esquemas de optimización numéricos.

Cabe mencionar que que los semi planos utilizados para generar las bifurcaciones en cada nodo del arbol no es necesario que sean de la forma $\{ x \in X | x_j \geq s\}$, tambien es posible considerar combinaciones lineales de las características. Esto por lo general puede mejorar la precisión pero se pierde por otro lado interpretabilidad del modelo.

**Ejercicio**

1. Implemente un modelo basado en árboles para predecir una separación ordinal de `SalePrice` en la base `HousePricing` usando las variables de interés utilizadas en la sección anterior. Recuerde generar un esquema que evite el sobreajuste del modelo.

### Métodos envolventes de Selección de Características

Esta clase métodos de selección de caráterísticas utilizan un estimador externo y utilizan estrategias de búsqueda sobre suconjuntos de variables optimizando las métricas de rendimiento del estimador externo. Se puede utilizar cualquier método de aprendizaje en conjunción a este tipo de métodos. Dentro de las estrategias de búsqueda se encuentran:

* **Selección hacia adelante**: Se comienza con un arreglo vacío, se entrenan modelos de predicción usando conjuntos de una característica para seleccionar aquella que con mayor rendimiento. El proceso se repite de manera secuencial.

Para implementar este tipo de métodos se puede hacer uso de la librería `mlxtend` de Python, esta librería permite extender ciertas funcionalidades de Scikit-learn y por tanto es completamente compatible con sus clases. Se importa el método de selección hacia adelante

In [ ]:
df_train = pd.read_csv('df_train_v1.csv', header=[0,1], index_col=0)
df_train.head(3)

In [ ]:
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.preprocessing import FunctionTransformer, RobustScaler
from sklearn.preprocessing import PowerTransformer, PolynomialFeatures

from sklearn.compose import ColumnTransformer

from sklearn.pipeline import Pipeline

se define un objeto `Pipeline` para transformación de columnas

trabajaremos con un modelo basado en árboles sobre la base de datos de `HousePricing`. En primera instancia se tratan las variables numéricas, estas se estandarizan de manera robusta y se filtran según asimetria estadística. Las variables con mayor asimetria son transformadas según una transformación de potencia

In [ ]:
# Variables numericas
def custom_scaler(df):
    '''Escala y retorna objeto DataFrame'''
    scaler = RobustScaler()
    return pd.DataFrame(scaler.fit_transform(df), columns=df.columns)


def skew_selector(df, skew_val=0.75):
    '''Reconoce columnas asimetricas.
    
    Obs: recibe solo valores de df_train[numeric]
    
    '''
    return [
        df.loc[:, df.columns[df.skew() > skew_val]],
        df.loc[:, df.columns[df.skew() <= skew_val]]
    ]


def custom_power_transform(df_list):
    '''Escala solo aquellas columnas con alta asimetria.'''

    scaler = PowerTransformer(method='yeo-johnson')
    transformed = pd.DataFrame(scaler.fit_transform(df_list[0]),
                               columns=df_list[0].columns)
    intact = df_list[1]

    return pd.concat((transformed, intact),axis=1, sort=False)


scaler = FunctionTransformer(func=custom_scaler)
to_yj = FunctionTransformer(func=skew_selector)
num_transform = FunctionTransformer(func=custom_power_transform)

num_pipe = Pipeline(steps=[('estandariza',
                            scaler), ('select by skew',
                                      to_yj), ('normaliza', num_transform)])

Las variables categóricas y ordinales son codificadas

In [ ]:
# Variables ordinales
ordinals = df_train['ordinal']
ordinal_cat = [[*ordinals[col].unique()] for col in ordinals.columns]

ord_pipe = Pipeline(
    steps=[('ordinal', OrdinalEncoder(categories = ordinal_cat))])

# Variables categoricas
enc = OneHotEncoder(sparse=False)
cat_pipe = Pipeline(steps=[('OneHot', enc)])

Se agregar caracteristicas por interacción de grado 2 usando `PolynomialFeatures`

In [ ]:
interactions = PolynomialFeatures(degree=2,interaction_only=False ,include_bias=False)
extra_features = Pipeline(steps=[('poli_interactions', interactions)])

Se unen los objetos `Pipeline` definidos para producir un dataset unificado, para ello se utiliza `ColumnTransformer` del módulo `compose`.

In [ ]:
# Funcion auxiliar
def train_indexer(cols, t_c=df_train.columns):
    '''Genera columnas multinivel a partir de nombres de columna planos.'''

    set_to_tuple = set(*[cols])

    tuples = [i for i in t_c if set_to_tuple.intersection(set(i))]

    return tuples

Se define el proceso de preprocesamiento y obtención de características

In [ ]:
num_cols = train_indexer(df_train['numeric'].columns)
num_cols.remove(('numeric','SalePrice'))

cat_cols = train_indexer(df_train['categorical'].columns)
ord_cols = train_indexer(df_train['ordinal'].columns)

# Preprocesamiento segun tipo de dato
preprocessor = ColumnTransformer(
    transformers=[('prepr_num', num_pipe, num_cols), 
                  ('prepr_cat', cat_pipe, cat_cols), 
                  ('prepr_ord', ord_pipe, ord_cols)], n_jobs = -1)

# Se agregan las interacciones
trans_process = Pipeline(
    steps=[('prepr', preprocessor), ('interactions', extra_features)])

Se observa que el número de carácteristicas que genera nuestro flujo es casi el más del doble que la cantidad de observaciones que se poseen

In [ ]:
X = pd.DataFrame(trans_process.fit_transform(df_train))
X.shape

A continuación se diseña un esquema de filtrado de caracterísitcas. En primera instancia se filtran eliminando las variables con menor varianza cuya varianza acumulada sea inferior al 1% de la varianza total.

In [ ]:
vars_sorted = X.var(axis=1).sort_values()
variance_thr = vars_sorted[vars_sorted.cumsum() <= vars_sorted.sum()*.01].max()

Se procede a generar una selector de características basado en el umbral definido anteriormente

In [ ]:
from sklearn.feature_selection import VarianceThreshold

var_selector = VarianceThreshold(threshold=variance_thr)

var_selector.fit_transform(X).shape

**Ejercicio**

1. ¿Se puede hacer lo anterior usando `SelectPercentile`?

Posteriormente, se obtiene que de las 26334 variables generadas, solo 2778 (el 10% aproximadamente) aportan el 99% de la varianza total. A continuación se filtran características según distintos coeficientes de correlación al comparar con la variable de respuesta, se filtran además según información mutua. 

In [ ]:
from sklearn.feature_selection import SelectPercentile, mutual_info_regression
from scipy.stats import kendalltau, spearmanr, pearsonr

# Se define un decorador para que las metricas de scipy sean compatibles
def mask(func):
    '''Decorador especfico para compatibilizar scipy.stats con dataframes.'''
    def wrapper(X, y, func=func):
        '''Toma la funcion func y le permite operar sobre la columnas de X.'''

        res = [func(X[:, i], y)[0] for i in range(X.shape[1])]
        return np.array(res)

    return wrapper


# Se decoran las funciones de scipy
kendalltau = mask(kendalltau)
pearsonr = mask(pearsonr)
spearmanr = mask(spearmanr)

# Se opera sobre X e y
y_train = df_train['numeric'].SalePrice.copy()

# Se declaran los selectores
mi_selector = SelectPercentile(score_func=mutual_info_regression,
                               percentile=20)

kendall_selector = SelectPercentile(score_func=kendalltau, percentile=20)

spearman_selector = SelectPercentile(score_func=spearmanr, percentile=20)

pearson_selector = SelectPercentile(score_func=pearsonr, percentile=20)

Con los selectores definidos anteriormente se genera un objeto `FeatureUnion`, este permite concatenar los resultados de cada selector

In [ ]:
from sklearn.pipeline import FeatureUnion

selectors = FeatureUnion(
    transformer_list=[('mutual info', mi_selector), 
                      ('Kendall corr', kendall_selector), 
                      ('Spearman corr',spearman_selector),
                      ('pearson corr',pearson_selector)]
                      , n_jobs=-1)

el proceso anterior se unifica en un objeto `Pipeline` que incorpora el proceso de filtrado por varianza

In [ ]:
filter_pipe = Pipeline(steps=[('var filter', var_selector),
                              ('corr mi selectors', selectors)]
                      )

Se ajusta el proceso de filtrado y se remueven las columnas duplicadas

In [ ]:
X_filtered = pd.DataFrame(filter_pipe.fit_transform(X,y_train))
X_filtered = X_filtered.T.drop_duplicates().T
X_filtered.shape

In [ ]:
X_filtered = pd.read_csv('X_filtered.csv', index_col=0)

In [ ]:
X_filtered.to_csv('X_filtered.csv')

Donde se observa que han ido seleccionadas 916 variables del total. Sobre esta cantidad de variables se realiza un esquema de **selección hacia adelante** utilizando `mlextend`. 

In [ ]:
from mlxtend.feature_selection import SequentialFeatureSelector as sfs
from sklearn.tree import DecisionTreeRegressor
import pickle

n_cols = (7,20)

# Se inicializa el regresor
with open('tree_model.pkl', 'br') as handler:
    tree_reg_sfs = pickle.load(handler)

# Se inicializa el buscador
forward_selector = sfs(tree_reg_sfs,
                       k_features=n_cols,
                       forward=True,
                       scoring='r2',
                       cv=15,
                       n_jobs=-1,
                       verbose=1)

Se procede a hacer la selección utilizando como métrica de rendimiento el puntaje `R2` y haciendo validación cruzada de 5 folds. Se inicializa el regresor con los hiperparametros encontrados anteriormente.

In [ ]:
#Lento
forward_selector.fit(X_filtered, y_train)

Se obtienen las caractetisticas seleccionadas por medio del método `.k_features_names_`

In [ ]:
X_filtered.columns[f_selector.k_feature_names_]

In [ ]:
# Abre el modelo generado -> sobreescribe f_selector
with open('forward_selector.pkl', 'br') as handler:
    f_selector = pickle.load(handler)

X_filtered.loc[:,map(str,f_selector.k_feature_names_)].head()

In [ ]:
# Guarda el modelo generado -> sobreescribe f_selector
import pickle

with open('forward_selector.pkl','bw') as handler:
    pickle.dump(forward_selector, handler)

Se repite el proceso de validación cruzada utilizando un modelo de arboles de desición, esta vez, sobre los datos con caracterísitcas filtradas:

In [ ]:
tree_reg = DecisionTreeRegressor(random_state=6202)

X_train_tree, X_test_tree, Y_train_tree, Y_test_tree = train_test_split(
    X_filtered.loc[:,map(str,f_selector.k_feature_names_)],
    y_train,
    test_size=.2)
#
grid = {
    'max_depth': range(7,13),
    'ccp_alpha': np.power(2, np.linspace(14, 16, num=2)),
    'min_samples_leaf': range(3, 10),
    'min_impurity_decrease': np.power(2, np.linspace(12.5, 13, num=3))
}


cv = GridSearchCV(tree_reg, param_grid=grid, cv=5, n_jobs=7)

cv.fit(X_train_tree, Y_train_tree)

tree_reg_best = cv.best_estimator_
tree_reg_best

Se comparan los resultados de test y train:

In [ ]:
train_res_gs = {
    'R2': tree_reg_best.score(X_train_tree, Y_train_tree),
    'rmse': rmse(tree_reg_best.predict(X_train_tree), Y_train_tree)
}

print('Resultados train: \n', train_res_gs, '\n')

test_res_gs = {
    'R2': tree_reg_best.score(X_test_tree, Y_test_tree),
    'rmse': rmse(tree_reg_best.predict(X_test_tree), Y_test_tree)
}

print('Resultados test: \n', test_res_gs)

Se obtiene una mejora notoria, con claros signos de una reducción en el sobre ajuste.

In [ ]:
# Se abre el mejor modelo
with open('sfs_tree_reg', 'br') as handler:
    tree_reg_best = pickle.load(handler)

In [ ]:
# Se guarda el mejor modelo
with open('sfs_tree_reg','bw') as handler:
    pickle.dump(tree_reg_best,handler)

* Selección hacia atras

Es análogo a la selección hacia adelante, en este procedimiento se comienza con todas las características y se elimina de manera secuancial aquellas con menores métricas de rendimiento.

**Ejercicio**

1. Utilice `SequentialFeatureSelector` para hacer una selección de variables hacia atrás.

2. Utilice la función `export_graphviz` para exportar un modelo entrenado de árbol de desición a un archivo en el archivo `tree.dot`. Abra el archivo usando el context manager `open(tree.dot)` en conjunción con la orden `read()`, almacene el resultado en la variable `tree_viz`. Utilice la clase `Source` de la librería `graphviz`  con la variable `tree_viz` como argumento. El resultado de estas acciones generan una visualización del árbol elegido. ¿Se puede obtener información sobre el comportamiento de las características?

* Selección por eliminiación reursiva (RFE)

Este método recibe un estimado externo, a diferencia de las técnicas anteriores, este estimador debe asociar pesos a sus características (ej: modelo lineal) o importancia de características (*feature importance*, ej: modelos basados en árboles). El procedimiento de selección de características consiste en *seleccionar hacia atrás* dado un conjunto inicial de características, a cada característica se le asocia un puntaje basándose en su coeficiente o importancia (proporcionados por el modelo externo) . Las caracteísticas con menor puntaje son eliminadas, el proceso se repite de manera recurrente. 

**Ejemplo**

Se efectúa una selección de caraterísticas por RFE sobre el datset `HousePricing` transformado, para ello se utiliza un modelo basado en árboles con los hiperparámetros ya encontrados. 

In [ ]:
from sklearn.feature_selection import RFECV

n_cols = 20

# Se inicializa el regresor
with open('tree_model.pkl', 'br') as handler:
    tree_reg_rfe = pickle.load(handler)

# Se inicializa el buscador
rfe_selector = RFECV(tree_reg_rfe,
                     min_features_to_select=n_cols,
                     scoring='r2',
                     cv=5,
                     n_jobs=7,
                     verbose=1,
                     step=129)

In [ ]:
rfe_selector.fit(X_filtered, y_train)

Se guarda la selección generada

In [ ]:
# Se abre --> reescribe rfe_selector
with open('rfe_selection.pkl','br') as handler:
    rfe_selector = pickle.load(handler)

In [ ]:
# Se guarda --> reescribe selector_rfe.pkl
with open('rfe_selection.pkl','bw') as handler:
    pickle.dump(rfe_selector,handler)

In [ ]:
tree_reg_rfe = DecisionTreeRegressor(random_state=6202)

X_train_tree, X_test_tree, Y_train_tree, Y_test_tree = train_test_split(
    X_filtered.loc[:, rfe_selector.support_],
    y_train,
    test_size=.2)

grid = {
    'max_depth': range(7,13),
    'ccp_alpha': np.power(2, np.linspace(14, 16, num=2)),
    'min_samples_leaf': range(3, 10),
    'min_impurity_decrease': np.power(2, np.linspace(12.5, 13, num=3))
}

cv = GridSearchCV(estimator = tree_reg_rfe, param_grid=grid, cv=5, n_jobs=-1, verbose = 1)

cv.fit(X_train_tree, Y_train_tree)

tree_reg_rfe_best = cv.best_estimator_
tree_reg_rfe_best

In [ ]:
train_res_gs = {
    'R2': tree_reg_rfe_best.score(X_train_tree, Y_train_tree),
    'rmse': rmse(tree_reg_rfe_best.predict(X_train_tree), Y_train_tree)
}

print('Resultados train: \n', train_res_gs, '\n')

test_res_gs = {
    'R2': tree_reg_rfe_best.score(X_test_tree, Y_test_tree),
    'rmse': rmse(tree_reg_rfe_best.predict(X_test_tree), Y_test_tree)
}

print('Resultados test: \n', test_res_gs)

Donde se obtienen resultados similares a los métodos de selección anteriores.

In [ ]:
# Se abre el mejor modelo
with open('tree_reg_rfe.pkl','br') as handler:
    tree_reg_rfe_best = pickle.load(handler)

In [ ]:
# Se guarda el mejor modelo para las características seleccionadas
with open('tree_reg_rfe.pkl','bw') as handler:
    pickle.dump(tree_reg_rfe_best, handler)

**Ejercicio**

La clase `TransformedTargetRegressor` del módulo `compose` permite entrenar regresores para los cuales se transforma el output según una función proporcionada (ej: `PowerTransformer`). Si se desea trabajar con búsqueda por grilla utilizando validación cruzada, es necesario entregar un diccionario de la forma:

```python

param_grid = {'regressor__HIPERPARAMETRO_1':value_range_1,
              'regresor__HIPERPARAMETRO_2':value_range_2,
              ...
              'regressor__HIPERPARAMETRO_n':value_range_n,
              }
```

En este caso `HIPERPARAMETRO_n` hace referencia a un hiperparámetro que se desea inspeccionar en el rango de valores `range_n` usando búsqueda por grilla `GridSearchCV`. 

1. Genere un modelo que haga una transformación sobre `y_train` cambiando su distribución a una distribución normal. Utilice un modelo basado en árboles de regresión, haga una búsqueda por grilla en al menos 3 hiperparámetros. Considerando que se seleccionaron características en función de la variable no transfromada, ¿espera que haya un mejor rendimiento en este modelo?

**Obs**: Observe que un objeto tipo `TransformedTargetRegressor` poseen el atributo `regressor`. Para modificar los hiperparámetros de tal atributo (cuyo valor es un objeto `Estimator`) dentro de un objeto `GridSearchCV` se debe hacer uso de una notación especial, que consiste en generar diccionarios de la forma `regressor__HIPERPARAMETRO_n:value_n`. Si no se utiliza esta notación, el buscador `GridSearchCV` intentará modificar los hiperparámetros de el transformador `TransformedTargetRegressor` y no del regresor de interés.

## Reducción de Dimensionalidad

Otra manera de seleccionar características es por medio de técnicas proyectivas o transformaciones desde espacios de mayor dimensión a otros de menor dimenisón. Dentro de las ténicas clasicas en este apartado, se tiene el analísis de componentes principales **PCA** por sus siglas en ingles.

### PCA

Este método, a grandes rasgos, consiste en ajustar un elipsoide $L$-dimensional sobre el conjunto de datos. Los ejes del elipsoide que son grandes en magnitud indican una dirección en la que hay gran variabilidad en los datos, en contraste, los ejes pequeños indican poca variabilidad. Considerando que los ejes de un elipsoide $L$-dimensional generan una base ortogonal para el espacio vectorial en el cual está contenida, al usar como representación los datos proyectados sobre los ejes con mayor magnitud, es posible construir una transformación de los datos que conserve gran parte de la variabilidad, pero que tenga menor dimensión.

Para aplicar este método es necesario que los datos tengan media 0 en cada componente, es decir, el conjunto de datos esté centrado respecto a $\vec{0}$. El procedimiento es sensible a la escala de los datos, pero no hay consenso general respecto a cuál es el mejor escalamiento para obtener una representación óptima.

Sea un conjunto de datos $\left \{ x_i \right \}_{i=1}^N$, los datos se organizan en una matriz $X = \left ( x_1,x_2,\ldots,x_N \right )^T \in R^{N \times L}$ con media empírica 0 en cada una de sus columnas.

La idea es encontrar una dirección tal que proyectados los datos sobre esta, maximice la varianza, es decir, se busca $w_1$ tal que:

\begin{equation}
w_1 = \underset{\left \| w \right \| = 1}{\arg\max} \hspace{1mm} \sum_{i=1}^N (x_i^T w)^2
\end{equation}

Escrito de forma matricial el problema es:

\begin{equation}
w_1 = 
\underset{\left \| w \right \| = 1}{\arg\max} \hspace{1mm} \left \| Xw \right \|^2 =
\underset{\left \| w \right \| = 1}{\arg\max} \hspace{1mm} w^T X^T X w = 
\underset{}{\arg\max} \hspace{1mm} \frac{w^T X^T X w}{w^T w}
\end{equation}

Un resultado clásico para matrices semi definidas positivas como $X^T X$, es que la cantidad $\left \| Xw \right \| $ está acotada por $\lambda_1$, el mayor valor propio de $X^T X$ y esto se alcanza cuando $w$ es igual al vector propio asociado a $\lambda_1$.

Ya obtenido el primer eje principal, para obtener el $k$-ésimo se procede substrayendo las $k-1$ componentes principales ya extraídas de $X$.

\begin{equation}
\hat{X}_k = X - \sum_{i=1}^{k-1} X w_i^T
\end{equation}

Por lo tanto la $k$-ésima componente principal $w_k$ es la solución del problema:

\begin{equation}
w_k = 
\underset{\left \| w \right \| = 1}{\arg\max} \hspace{1mm} \left \| \hat{X}_kw \right \|^2 =
\underset{}{\arg\max} \hspace{1mm} \frac{w^T \hat{X}_k^T \hat{X}_k w}{w^T w}
\end{equation}

La solución a este problema resulta ser el $k$-ésimo vector propio de la matriz $X^T X$.

Finalmente encontrar las componentes principales se reduce a diagonalizar la matriz de covarianza de los datos, es decir, una descomposición de la forma:

\begin{equation}
X^T X = W \Lambda W^T
\end{equation}


Donde $W = \left ( w_1, \ldots, w_L \right ) $ es la matriz que tiene los vectores propios de $X^T X$ en sus columnas y $\Lambda$ es una matriz diagonal con los valores propios ordenados de forma decreciente. De esta forma la descomposición en componentes principales de los datos queda dada por:

\begin{equation}
T = X W \in \mathbb{R}^{N \times L}
\end{equation}

Para reducir los datos a $K$ dimensiones, se toma la matriz $W_K = \left ( w_1, \ldots, w_K \right )$ con los $K$ vectores propios con mayor valor propio y se proyectan los datos sobre estos:


\begin{equation}
T_K = X W_K  \in \mathbb{R}^{N \times K}
\end{equation}

Para aplicar este método de manera computacional se hace uso de la clase `PCA` de módulo `decomposition` de Scikit-learn.

**Ejemplo**

Se utilizan las componentes filtradas por varianza y correlación.

In [ ]:
X_filtered = pd.read_csv('X_filtered.csv', index_col=0)
X_filtered.shape

Se puede ver que aunque el conjunto de datos tenga 916 caraterísticas, el 99% de la varianza del dataset se puede explicar con 133 variables proyectadas

In [ ]:
from sklearn.decomposition import PCA

n_components = 133
pca = PCA(n_components = n_components)

X_pca = pca.fit_transform(X_filtered)

pca.explained_variance_ratio_.cumsum()[-2:]

se estudia el aporte de varianza por componente

In [ ]:
explained_variance = pca.explained_variance_ratio_

plt.figure(0)
plt.plot(explained_variance)

plt.figure(1, figsize=(15,6))

for i in range(10):
    plt.subplot(2,5,i+1)
    plt.plot(pca.components_[i])
    plt.title(f'C. principal {i+1}')

se observan proyecciones de 2 dimensiones basadas en las 3 primeras componentes principales.

In [ ]:
n_components = 133
pca = PCA(n_components = n_components)

bins = pd.cut(y_train,2)

X_pca = pca.fit_transform(X_train)
fig, ax = plt.subplots(3,1, figsize=[9,15])

sns.scatterplot(X_pca[:,0], X_pca[:, 1], hue=bins,ax=ax[0])
ax[0].set_title('Comp. 1 vs Comp. 2')

sns.scatterplot(X_pca[:,0], X_pca[:, 2], hue=bins,ax=ax[1])
ax[1].set_title('Comp. 1 vs Comp. 3')

sns.scatterplot(X_pca[:,1], X_pca[:, 2], hue=bins,ax=ax[2])
ax[2].set_title('Comp. 2 vs Comp. 3')

### K-PCA

El presente método consiste en una extensión de PCA, que explotando el llamado "truco del kernel" permite realizar reducción de dimensionalidad en un espacio de características transformado. Esto permite capturar de mejor forma la estructura de datos que presentan comportamiento no lineal, algo que no es capás de llevar a cabo la versión clásica de PCA, limitado por la naturaleza de las operaciones lineales que componen el procedimiento.

El método se apoya en la utilización de una función no lineal que mapea el conjunto de datos original a un espacio de características, con dimensión usualmente mayor:

\begin{equation}
\phi: \mathbb{R}^L \rightarrow \mathbb{R}^M
\end{equation}

la intuición de esto es que un conjunto de datos que no es linealmente separable, puede ser separable en un espacio de mayor dimensión utilizando una transformación.


Sea un conjunto de datos $\left \{ x_i \right \}_{i=1}^N \subset \mathbb{R}^L $, los desarrollos realizados en la sección anterior pueden ser reproducidos en el espacio transformado por la aplicación $\phi$. Por simpleza se asume que la media empírica de los datos es 0, es decir:

\begin{equation}
\frac{1}{N} \sum_{i=1}^N \phi(x_i) = 0
\end{equation}

de no ser así se puede sustraer la media empírica a la transformación para corregir esto.

La matriz de covarianza de los datos transformados es:

\begin{equation}
C = \frac{1}{N} \sum_{i=1}^N \phi(x_i) \phi(x_i)^T
\end{equation}

Para $k = 1, \ldots, M$ el problema de auto valores para $C$ es:

\begin{align}
C v_k =& \lambda_k v_k \\
\frac{1}{N} \sum_{i=1}^N \phi(x_i) \{ \phi(x_i)^T v_k \} =& \lambda_k v_k
\end{align}

esto puede ser re escrito como:

\begin{equation}
v_k = \sum_{i=1}^N a_{ki}\phi(x_i)
\end{equation}

Al juntar las 2 ecuaciones anteriores se obtiene:

\begin{equation}
\frac{1}{N} \sum_{i=1}^N \phi(x_i)  \phi(x_i)^T  \sum_{j=1}^N a_{kj}\phi(x_j) = \lambda_k \sum_{i=1}^N a_{ki}\phi(x_i)
\end{equation}

Se define la función de Kernel:

\begin{equation}
k(x_i, x_j) = \phi(x_i)^T \phi(x_j)
\end{equation}

se multiplica la ecuación anterior por $\phi(x_l)^T$, obteniendo:

\begin{equation}
\frac{1}{N} \sum_{i=1}^N k(x_l,x_i)  \sum_{j=1}^N a_{kj} k(x_i,x_j) = \lambda_k \sum_{i=1}^N a_{ki}k(x_l,x_i)
\end{equation}

Llevando esto a notación matricial:

\begin{equation}
K^2 a_k = \lambda_k N K a_k
\end{equation}

Donde $K_{i,j} = k(x_i, x_j)$ y $a_k = ( a_{k1}, \dots, a_{kN}) ^T \in \mathbb{R}^N$, el cual puede ser resuelto en:

\begin{equation}
K a_k = \lambda_k N a_k
\end{equation}

y finalmente la transformación en componentes principales será:

\begin{equation}
y_k(x) = \phi(x)^T v_k = \sum_{i=1}^N a_{ki} k(x, x_i)
\end{equation}

Todo este preámbulo fue hecho para que fuera visible el hecho que en esta construcción se pasó de un problema de valores propios con dimensión $M$ (espacio de características trasnsformado) a uno con dimensión $N$ ($a_k$ tiene dimensión igual a la cantidad de muestras). En este sentido este método puede ser computacionalmente prohibitivo si la cantidad de muestras es demasiado grande, pero se evita tener que evaluar la tranformación $\phi$ y calcular el producto punto, por la evaluación del kernel $k$ en el conjunto de entrenamiento.

**Ejemplo**

Se puede implementar el método K-PCA usando el objeto `KernelPCA` del módulo `decomposition` de Scikit-learn

In [ ]:
from sklearn.decomposition import KernelPCA

comps = 100
gamma = 1e-4
kpca = KernelPCA(n_components = comps, kernel = 'rbf', gamma = gamma, n_jobs=-1)

X_kpca = kpca.fit_transform(X_filtered)

Se estudia la varianza explicada por las nuevas características

In [ ]:
# Valores propios asociados
explained_variance = kpca.lambdas_

plt.figure(0)
plt.plot(explained_variance[:10])

plt.figure(1, figsize=(17,7))

for i in range(10):
    plt.subplot(2,5,i+1)
    plt.plot(kpca.alphas_[i])
    plt.title(f'C. principal {i+1}')

In [ ]:
fig, ax = plt.subplots(3,1, figsize=[9,15])

bins = pd.cut(y_train,2)
sns.scatterplot(X_kpca[:,0], X_kpca[:, 1], hue=bins,ax=ax[0])
ax[0].set_title('Comp. 1 vs Comp. 2')

sns.scatterplot(X_kpca[:,0], X_kpca[:, 2], hue=bins,ax=ax[1])
ax[1].set_title('Comp. 1 vs Comp. 3')

sns.scatterplot(X_kpca[:,1], X_kpca[:, 2], hue=bins,ax=ax[2])
ax[2].set_title('Comp. 2 vs Comp. 3')

Con lo anterior se puede ver que las componentes puede discriminar de cierta forma entre 2 niveles de precio, por lo menos para la componente 1 vs 2.

### LDA

**LDA** (**L**inear **D**discriminant **A**nalysis) en un método de reducción de dimensión lineal que aprovecha etiquetas en una base de datos, por este motivo (y a diferencia de (K)PCA) es un método supervisado.

El método se enfoca en maximizar la diferencia entre los puntos proyectados de clases distintas y minimizar la distancia entre puntos de la misma clase, a la vez.

Sea $\left \{ x_i \right \}_{i=1}^N \subset \mathbb{R}^L $ conjuntos de puntos distribuidos en $K$ clases distintas, $\left \{ y_i \right \}_{i=1}^N \subset \{ 1,\ldots,K \}^L $ es la variable que indica a que clase perteneca cada punto.

Se define

\begin{align}
C_i =& \{x_j\}_{j: y_j=i}\\
\mu_i =& \frac{1}{|C_i|} \sum_{x\in C_i} x \\
\mu =& \frac{1}{N} \sum_{i=1}^N x_i
\end{align}

adicionalmente:

\begin{align}
\Sigma_b =& \sum_{i=1}^K (\mu_i - \mu)(\mu_i - \mu)^T \\
\Sigma_w =& \sum_{i=1}^K \sum_{x \in } (x - \mu_i)(x - \mu_i)^T
\end{align}

Donde $\Sigma_b$ representa una medida de la variabilidad entre clases distintas y $\Sigma_w$ la dispersión entre la misma clases.

La función que se optimiza en este caso es:

\begin{equation}
J(W) = \frac{|W \Sigma_b W|}{|W \Sigma_w W|}
\end{equation}

La solución a este problema corresponde a uno de valores propios y su formulación es la siguiente:

\begin{equation}
S_b^{1/2}S_w^{-1}S_b^{1/2} v = \lambda v
\end{equation}

**Ejemplo**

Se utiliza LDA sobre los datos anteriores por medio de la clase `LinearDiscriminantAnalysis` del módulo `discriminant_analysis` de Scikit-learn.

In [ ]:
X_filtered.shape

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

bins = pd.cut(y_train,10)
labels = [*map(str,bins)]

lda = LDA(n_components=9)
X_lda = lda.fit_transform(X_filtered, labels)


plt.figure(0)
plt.plot(lda.explained_variance_ratio_)

plt.figure(1, figsize=(15,6))

for i in range(6):
    plt.subplot(2,3,i+1)
    plt.plot(lda.coef_[i])
    plt.title(f'C. principal {i+1}')

In [ ]:
fig, ax = plt.subplots(3,1, figsize=[9,15])

sns.scatterplot(X_lda[:,0], X_lda[:, 1], hue=bins,ax=ax[0])
ax[0].set_title('Comp. 1 vs Comp. 2')

sns.scatterplot(X_lda[:,0], X_lda[:, 2], hue=bins,ax=ax[1])
ax[1].set_title('Comp. 1 vs Comp. 3')

sns.scatterplot(X_lda[:,1], X_lda[:, 2], hue=bins,ax=ax[2])
ax[2].set_title('Comp. 2 vs Comp. 3')

Se observa que al incluirse información sobre la variable de respuesta, se pueden reconocer ciertos patrones de su distribución en menores dimensiones.

In [ ]:
with open('lda_dim.pkl', 'bw') as handler:
    pickle.dump(lda,handler)

### t-SNE (t-distributed stochastic neighbor embedding)

PCA se basa en hipótesis sobre la varianza global de los datos analizados para proyectar de manera lineal en una cantidad menor de dimensiones, KPCA flexibiliza este concepto permitiendo proyecciones no lineales. Por su parte, LDA toma proyecciones lineales que toman en cuenta ciertas estructuras de varianza local, para ello hace uso de etiquetas en problemas de clasificación. 

t-SNE es una técnica que permite reducir la dimensionalidad tomando en cuenta estructuras locales y globales de similitud. En términos generales, el algoritmo busca codificar la similitud entre puntos de alta dimensión por medio de un kernel gaussiano, una vez medidas las similitudes entre puntos, se procede a generar una estimación de la densidad de probabilidad conjunta de estos, finalmente se genera una densidad de probabilidad en bajas dimensiones que se aproxime a la estimada (en altas dimensiones) en función de la divergencia de Kullback-Leibler.

La formulación teórica pasa por definir $X = \left \{ x_i \right \}_{i=1}^N \subset \mathbb{R}^M $ como una colección de puntos. Luego,el primer paso de t-SNE consiste en calcular la probabilidad $p_{ij}$, la cual es proporcional a la similitud entre los puntos $x_i$ y $x_j$, de la siguiente forma:

\begin{equation}
p_{j|i} = \frac{\exp(-\left \| x_i - x_j \right \|^2/2\sigma_i^2)}{\sum_{k\neq i} \exp(-\left \| x_i - x_k \right \|^2/2\sigma_i^2)}
\end{equation}

Se define:

\begin{equation}
p_{ij} = \frac{p_{j|i} + p_{i|j}}{2N}
\end{equation}

adicionalmente si $i=j$ entoces $p_{ii} = 0$.

El parámetro que falta por definir es $\sigma_i$, la varianza de cada Gaussiana centrada en $x_i$, no es claro que exista un $\sigma_i$ óptimo para cada punto en la base de datos, este valor va a variar dependiendo de que tan densamente pobladas sean las regiones en el espacio. t-SNE utiliza busqueda binaria para encontrar in $\sigma_i$ que produzca una distribución $P_i$ con una **perplejidad** (perplexity) igual a la definida por el usuario, en este sentido el algoritmo recibe como entrada la **perplejidad** y luego determina el respectivo $\sigma_i$.

El valor de la **perplejidad** es:

\begin{equation}
Perp(P_i) = 2^{E(P_i)}
\end{equation}

Donde $E(P_i)$ corresponde a la entropía de Shannon:

\begin{equation}
E(P_i) = -\sum_j p_{j|i} \log p_{j|i}
\end{equation}

t-SNE apunta a buscar un representación $d$-dimensional $Y = \left \{ y_i \right \}_{i=1}^N \subset \mathbb{R}^d$ que refleje las similaridades $p_{ij}$ de la mejor forma posible. Para esto se usa $q_{ij}$ el cual mide similitud entre $y_i$ e $y_j$, la definición es:

\begin{equation}
q_{ij} = \frac{(1 + \left \| y_i - y_j \right \|^2)^{-1}}{\sum_{k\neq i} (1 + \left \| y_i - y_k \right \|^2)^{-1}}
\end{equation}

Donde acá se utiliza una ditribución-t con 1 grado de libertad, al ser esta de "colas pesadas", los puntos muy distintos entre si quedarán alejados en la representación de baja dimensión. Adicionalmente se define $q_{ii}$ = 0.

La ubicación de los puntos $y_i$ será determinada minimizando la divergencia de Kullback-Leibler de la distribución P respecto a Q, esto es:

\begin{equation}
C = D_{KL}(P||Q) = \sum_{i \neq j} p_{ij} \log \frac{p_{ij}}{q_{ij}}
\end{equation}

Esta forma luego es minimizada respecto a las variables $y_i$ utilizando descenso de gradiente. Dada su formulación, se pueden obtener distintas representaciones sobre los mismos parámetros, en el siguiente articulo, se discuten ciertos aspectos de interás al trabajar con t-SNE [link](https://distill.pub/2016/misread-tsne/).

**Ejemplo**

Se busca una representación de 3 dimensiones para el datset con características filtradas

In [ ]:
from sklearn.manifold import TSNE

tsne = TSNE(n_components=3, perplexity=50)
X_tsne = tsne.fit_transform(X_filtered)

In [ ]:
fig, ax = plt.subplots(3, 1, figsize=[9, 15])

sns.scatterplot(X_tsne[:, 0], X_tsne[:, 1], hue=bins, ax=ax[0])
ax[0].set_title('Comp. 1 vs Comp. 2')

sns.scatterplot(X_tsne[:, 0], X_tsne[:, 2], hue=bins, ax=ax[1])
ax[1].set_title('Comp. 1 vs Comp. 3')

sns.scatterplot(X_tsne[:, 1], X_tsne[:, 2], hue=bins, ax=ax[2])
ax[2].set_title('Comp. 2 vs Comp. 3')

**Obs:** Las técnicas de reducción de dimensionalidad vista pueden ser utilizadas en conjunto con técnicas de clustering para obtener nuevas características, esta nuevas variables pueden ser añadidas al dataset incial donde pueden ser estudiadas con las técnicas ya vistas. Se debe ser muy cauteloso en este procedimiento pues los clusters obtenidos luego de reducir la dimensionalidad pueden no ser interpretables desde la óptica original del problema (ej: t-sne + clustering).

#### Métodos Básicos de Estimación

Ya hemos descrito los aspectos fundamentales en manejo de características, ya sea en la generación de nuevas variables como en la reducción de estas. A continuación se realizará una revisión general de algoritmos de aprendizaje y ciertos aspectos relacionados a los procesos de entrenamiento.

### Métodos de ensamblaje

Uno de los problemas de los arboles de desición es la inestabilidad de sus estructuras. Pequeñas modificaciones en el conjunto utilizado para el proceso de entrenamiento pueden generar cambios radicales en el proceso de bifurcaciones, esto hace que la interpretabilidad de estos métodos sea precaria para ciertos contextos.

La principal razón de la instabilidad de los arboles es la naturaleza jerárquica del procedimiento. Modificaciones en la bifurcación de nodos cercanos a la raíz genera diferencias que se propagan de forma acumulativa, de esta forma la topología del arbol puede cambiar radicalmente a lo largo del proceso al variar el conjunto de datos.

Una forma de lograr modelos más robustos es por medio de **ensamblaje**, esta práctica consiste en combinar estimadores base, de manera tal, que su efecto combinado mejore la capacidad de generalización. Por lo general, los métodos de ensamblaje conllevan esquemas de votación entre modelos de alta varianza para prevenir predicciones fuera de rango y sobreajuste, por otra parte se pueden utilizar técnicas de *levantamiento* (*boosting*) para mejorar el rendiento de un conjunto de modelos *débiles* o menos expresivos (*weak learners*).

**Ejemplo**

El modelo más básico de ensamblaje es **voto de la mayoria**, este funciona en un contexto de clasificación. Acá se tienen $n$ modelos $\lbrace h_1, \ldots, h_n \rbrace$, donde para un valor de entrada $x$, la clasificación se hace según la moda sobre los valores $h_i(x)$ para todo $i = 1, \ldots, n$. Consideremos un ejemplo de clasificación binaria, para que este tipo de modelos funcione, se requiere que el error de cada modelo $h_i$ sea menor que el 50% (mejor que adivinar al azar). 

Se puede calcular la probabilidad $P(m)$ de hacer una predicción errónea dado que $m$ clasificadores (con $m > \lceil {n/2} \rceil $) predicen la misma etiqueta. Para esto, se define el conjunto $\mathcal{C}_m$, consistente de todos los $n \choose m$ subconjuntos de $\lbrace 1, \ldots, n \rbrace$ con tamaño $m$. Así, si la probabilidad de que el clasificador $h_i$ falle en predicción es de $\varepsilon_i$, se puede modelar la distribución de aciertos y fallos como una variable aleatoria $Ber(\varepsilon_i)$. Utilizando la ley de probabilidades totales se puede llegar entonces al siguiente resultado:

$$
P(m) = \sum_{c \in C_m} P(m |\text{solo fallan } h_i  \forall i \in c) P(\text{solo fallan } h_i \forall i \in c)
$$

Dado que se tiene voto por mayoria y $|c| = m > \lceil {n/2} \rceil$  la expresión anterior se reduce a

$$
P(m) = \sum_{c \in C_m} P(\text{solo fallan } h_i \forall i \in c)
$$

Que corresponde a

$$
P(m) = \sum_{c \in C_m} \prod_{j \in c} \varepsilon_j \prod_{k \in \lbrace 1, \ldots, n \rbrace / c} (1-\varepsilon_k)
$$

Finalmente la probabilidad de que el ensamblaje falle es de 

$$
\varepsilon_{ens} = \sum_{m > \lceil n/2 \rceil} P(m) = \sum_{m > \lceil n/2 \rceil} \sum_{c \in C_m} \prod_{j \in c} \varepsilon_j \prod_{k \in \lbrace 1, \ldots, n \rbrace / c} (1-\varepsilon_k)
$$

Consideremos por ejemplo que se tienen $n=15$ modelos y $\varepsilon = 0.3$ para cada uno de los $n$ modelos. Según lo anterior, se tiene que el error de ensamblaje es:

$$
\varepsilon_{ens} = \sum_{m = 8}^{15} {11 \choose m} 0.3^m(1-0.3)^{(15-m)} = 0.05
$$


In [ ]:
from scipy.special import binom

n = 15
eps = .3

eps_ens = np.sum([binom(n,m)*(eps**(m))*((1- eps)**(n-m)) for m in range(int(np.ceil(n/2)),n+1)])
eps_ens

Esta técnica sugiere generar un gran número de mododelos de clasificación y unificarlos por la moda de sus respuesta. Cuando se trabaja con clasificadores probabílisiticos se puede generar un modelo de ensamblaje por **voto de mayoria suave**. En este tipo de modelo, se genera una combinanación convexa sobre las probabilidades de predicción de etiqueta entregadas por los modelos a ensamblar, posteriormente, se clasifica según la probabilidad más alta. En términos matemáticos, si $p_{i,j}$ es la probabilidad de predicción para la etiqueta $j$, entregada por el modelo $i$, para los pesos no negativos $w_i$, tales que $\sum_i^n w_i = 1$, la predicción se esamblaje viene dada por:
$$
\hat{y}=\arg \max _{j} \sum_{i=1}^{n} w_{i} p_{i, j}
$$

**Ejercicio**

1. Utilice `VotingClassifier` del módulo `sklearn.ensemble` para implementar voto por mayoría en un problema de clasificación simple.

#### Bagging

Cómo fue mencionado recientemente, un problema estructural de los métodos basados en arboles es su alta varianza respecto a las predicciones. Una idea razonable puede ser utilizar una gran cantidad modelos basados en árboles en conjunto con algún método de voto por mayoría. Dicha idea se puede elaborar utilizando el técnicas de sub-muestreo de datos (*boostraping*) y una ligera modificación del voto por mayoría conocida como **bagging**. 

En términos simples, el sub muestreo o *bootstraping* consiste en una muestra de tamaño $n$ obtenida **con reemplazo** a partir de un conjunto de datos original $\mathcal{D}$ con $|\mathcal{D}| = n$. Esto produce que algunos ejemplos de entrenamiento se dupliquen en cada sub-muestra, por consiguiente, existen también observaciones que no aparecen en en todas las sub-muestras. 

**Ejercicio**

1. Demuestre que de manera asintótica sobre el tamaño de la muestra, si se eligen $n$ observaciones de manera uniforme y con reemplazo (boostrap) entonces la submuestra generada tendrá alproximadamente el 63.2% de los valores únicos. Consecuentemente, habrá un 37.2% de observaciones que no aparecerán en la sub-muestra. *Hint*: Relacione la probablidad de que una muestra no sea seleccionada con el número $e$. 

Un esquema de bagging consiste en generar $m$ sub-muestras por boostrap, luego para cada muestra se entrena un modelo de predicción $h_i$, finalmente el modelo final de predicción corresponde a un ensamblaje de voto por mayoria (promedio en regresión) de los modelos generados.

### Boosting:

El concepto de **boosting** es una de las hearramientas más poderosas introducidas en los últimos 20 años. La motivación de este método de ensamblaje consiste en combinar el resultado de varios predictores "débiles" (ej: un clasificador que se relaciona débilmente con las etiquetas reales) para construir un clasificador con mucha mayor eficacia y presición. En general existen 2 tipos de boosting: boosting adaptativo y boosting por gradiente.

El proceso de boosting es iterativo y consiste en asignar pesos a cada elemento del conjunto de entrenamiento. Estos pesos se asignan de manera secuencial y su calculo se basa en los errores hechos por los modelos débiles o base. En términos generales, el algoritmo de boosting se reduce a:

1. Inicializar un vector de pesos uniforme, a cada observación del conjunto de entrenamiento se le asigna un peso.

2. Iterar:
    1. Entrenar un modelo débil $h_m$ minimizando un funcional de error $J_m$. Acá cada muestra pondera un error de predicción proporcional a su peso asignado.
    
    2. Aumentar el peso de las observaciones mal clasificadas.

3. Se genera un esquema de voto por mayoría en los predictores entrenados. 

**AdaBoost (Boosting Adaptativo)**

Para el caso de clasificación binaria, el algoritmo AdaBoost implementa el esquema anterior de la siguiente forma:

* Inicializa k: número de iteraciones.
* Inicializa $\mathcal{D}$: conjunto de entrenamiento $\mathcal{D}=\left\{ \left( \mathbf{x}^{[1]}, y^{[1]} \right), \ldots, \left( \mathbf{x}^{[n]}, y^{[n]}\right)  \right\}$.
* Inicializa $w_{1}(i)=1 / n, \quad i=1, \ldots, n, \quad \mathbf{w}_{1} \in \mathbb{R}^{n}$

1. Itera desde $r=1$ hasta $k$:
    1. Para todo $i$: $\mathbf{w}_r(i) := w_r(i)/{\sum_j w_r(j)}$
    2. Entrena $h_r$ modelo en $\mathcal{D,\mathbf{w_r}}$
    3. $\varepsilon_r = \sum_i w_r(i) \mathbf{1}\left(h_{r}(i) \neq y_{i}\right)$, si $\varepsilon_r > 1/2$ Terminar proceso.
    4. $\alpha_r := 1/2 \log[(1-\varepsilon_r)/\varepsilon_r]$
    5. 
    $$
    w_{r-1}(i):=w_{r}(i) \times\left\{\begin{array}{ll}\mathrm{e}^{-\alpha_{r}} & \text { if } h_{r}\left(\mathrm{x}^{[l]}\right)=y^{[i]} \\ \mathrm{e}^{\alpha_{r}} & \text { if } h_{r}\left(\mathrm{x}^{[i]}\right) \neq y^{[i]}\end{array}\right.
    $$
    
Se predice según $h_{k}(\mathbf{x})=\arg \max _{j} \sum_{r} \alpha_{r} \mathbf{1}\left[h_{r}(\mathbf{x})=j\right]$.

**Ejercicio**

Generalice el algoritmo anterior para trabajar en clasificación con $c$ clases. Para ello tendrá que modificar $\varepsilon_r$ y $\alpha_r$. ¿Como se puede generalizar este esquema a regresión?

**Ejemplo**

Se aplica AdaBoost sobre la reducción de dimensionalidad propuesta por LDA sobre el conjunto de datos filtrado

In [ ]:
with open('lda_dim.pkl', 'br') as handler:
    lda_dim = pickle.load(handler)

In [ ]:
from sklearn.ensemble import AdaBoostRegressor

ab = AdaBoostRegressor(base_estimator=DecisionTreeRegressor(),
                       n_estimators=600)

X_train_tree, X_test_tree, Y_train_tree, Y_test_tree = train_test_split(
    lda_dim.transform(X_filtered), y_train, test_size=.2)

grid = {
    'base_estimator__max_depth': range(7, 9),
    'base_estimator__ccp_alpha': np.power(2, np.linspace(14, 16, num=2)),
    'base_estimator__min_samples_leaf': range(3, 5),
    'base_estimator__min_impurity_decrease': np.power(2, np.linspace(12.5, 13, num=2))
    }

cv = GridSearchCV(ab, param_grid=grid, cv=5, n_jobs=-1, verbose=1)

cv.fit(X_train_tree, Y_train_tree)

ab_reg_best = cv.best_estimator_
ab_reg_best

In [ ]:
train_res_gs = {
    'R2': ab_reg_best.score(X_train_tree, Y_train_tree),
    'rmse': rmse(ab_reg_best.predict(X_train_tree), Y_train_tree)
}

print('Resultados train: \n', train_res_gs, '\n')

test_res_gs = {
    'R2': ab_reg_best.score(X_test_tree, Y_test_tree),
    'rmse': rmse(ab_reg_best.predict(X_test_tree), Y_test_tree)
}

print('Resultados test: \n', test_res_gs)

In [ ]:
with open('lda_ada_boost.pkl','bw') as handler:
    pickle.dump(ab_reg_best,handler)

**Gradient Boosting**

En contraste con AdaBoost, acá no se ajustan los pesos de las observaciones mal clasificadas. En su lugar, se optimiza una función de perdida suave (L2 por ejemplo) para un predictor débil. El output de este método es un modelo aditivo en base a un conjunto de predictores base, es decir, no se aplica votación. Así, para la función de pérdidia diferenciable $L(y,\hat{y})$, el algoritmo consiste en:

1. Inicializar un modelo constante $h_{0}(x)=\arg \min _{\gamma} \sum_{i=1}^{n} L\left(y_{i}, \gamma\right)$

2. Itera desde $m=1$ hasta $M$:
    1. Obtener los residuos
    $$
    r_{i m}=-\left[\frac{\partial L\left(y_{i}, F\left(x_{i}\right)\right)}{\partial F\left(x_{i}\right)}\right]_{F(x)=F_{m-1}(x)} \quad$ for $i=1, \ldots, n
    $$
    2. Entrenar un modelo base $h_m(x)$ sobre el conjunto $\left\{\left(x_{i}, r_{i m}\right)\right\}_{i=1}^{n}$ 
    3. Calcular los ponderadores de boosting $\gamma_m$ 
    $$
    \gamma_{m}=\underset{\gamma}{\arg \min } \sum_{i=1}^{n} L\left(y_{i}, F_{m-1}\left(x_{i}\right)+\gamma h_{m}\left(x_{i}\right)\right)
    $$
    4. Actualizar el modelo $F_{m}(x)=F_{m-1}(x)+\gamma_{m} h_{m}(x)$

3. Output $F_{M}(x)$

**Ejemplo**

Se utiliza un regresor basado en gradient boosting sobre los datos transformados por LDA. Este método esta basado en arboles como predictores débiles.

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor 

gb = GradientBoostingRegressor(n_estimators=600)

X_train_tree, X_test_tree, Y_train_tree, Y_test_tree = train_test_split(
    lda_dim.transform(X_filtered), y_train, test_size=.2)

grid = {
    'max_depth': range(7, 9),
    'ccp_alpha': np.power(2, np.linspace(14, 16, num=2)),
    'min_samples_leaf': range(3, 5),
    'min_impurity_decrease': np.power(2, np.linspace(12.5, 13, num=2))
    }

cv = GridSearchCV(gb, param_grid=grid, cv=5, n_jobs=-1, verbose=1)

cv.fit(X_train_tree, Y_train_tree)

gb_reg_best = cv.best_estimator_
gb_reg_best

In [ ]:
train_res_gs = {
    'R2': gb_reg_best.score(X_train_tree, Y_train_tree),
    'rmse': rmse(gb_reg_best.predict(X_train_tree), Y_train_tree)
}

print('Resultados train: \n', train_res_gs, '\n')

test_res_gs = {
    'R2': gb_reg_best.score(X_test_tree, Y_test_tree),
    'rmse': rmse(gb_reg_best.predict(X_test_tree), Y_test_tree)
}

print('Resultados test: \n', test_res_gs)

### Stacking 

El algoritmo de stacking consiste en ensamblar modelos basados en sus predicciones. Sobre tales predicciones se construye un *meta* modelos que las combina minimizando el error asociado. El algorimto de este método consiste en:

1. Input: Datos de entrenamiento $\mathcal{D} = \lbrace x_i, y_i \rbrace_{i=1}^{m}$
2. Output: Clasificador ensamblado $H$. 

Etapa inicial:
1. Entrena $T$ modelos $h_i$ base sobre $D$

Etapa secundaria:
1. Se construye un nuevo conjunto de datos:
$$
D_{h}=\left\{x_{i}^{\prime}, y_{i}\right\},\text{ donde }x_{i}^{\prime}=\left\{h_{1}\left(x_{i}\right), \ldots, h_{T}\left(x_{i}\right)\right\}
$$

Etapa final:

Entrenar $H$ sobre $D_{h}$ 

**Ejercicio**

1. Utilice la clase `StackingRegressor` de módulo `sklearn.ensemble` para entrenar un modelo de ensamblaje en regresión.

## Modelos Clasicos

## Selección de Caracterísitcas implicita